**First attempts**
Wednesday, February 26, 2020
2:03 PM
 
Got access to HPC via webCARS capacity clusters, simple ssh kevmoor@skybridge and kerb pass via git bash (which is just mingw).  May be good to switch to linux for local development.  Great that they are using slurm!  I may have also had to get access to matlab coder license via NILE.

Followed tutorial https://www.mathworks.com/videos/automatically-converting-matlab-code-to-c-code-96483.html compiled output on skybridge using:  cc main.cpp test_function.cpp test_function_terminate.cpp -o pleasework.  Would obviously use cmake for better compiling of more complex programs.  Didn't use srun to get slurm to give me actual compute time a resources.  Would need to do that, or sbatch for a auto batch run if we were running anything for real/substantial.

So, the matlab coder utility can output both source and executables, but were compiler issues right out of the box, which in my experience is typical for windows systems.  The code output for the simple function worked well enough for c and c++ output.  I turned off all of the overhead type things to make to code simpler and faster.

Now switching to the application compiler: I had issues trying to get it to create an installer with the runtime included.  I installed the runtime from NILE, but after a half hour of trying to get it to connect up, I stopped.  I'm sure it can be done, just isn't obvious to me.  The web installer works just fine and does output prebuilt executables as well, which run, but there is a lot of overhead time for it to start.  My simple function just output a few things to the terminal, so I couldn't just double click the executable, I had to run it from the terminal to see the output.

Now trying to run the version of OWENS I have through these tools:  Using the code generation readiness utility (right click a function and at the bottom), it said that the following were unsupported: fscanf, ismembc, str2num, warning, readtable. When I fixed those, and the syntax errors, it didn't like variable length arrays and made me pre-allocate them.

Ok, since I don't have a driver for OWENS with the proper input files yet, I switched to Brandon's blade optimization.  It had 889 unsupported Function calls, from regexp, to cd, to sscanf, to warning, to clc, and lots and lots of others.  So, I switched to the application compiler and it did compile (different than the C++ compilation, the application compiler requires the matlab runtime overhead).  But… it didn't run, error message says that Matlab compiler doesn't support modifying the search path, and I'd expect other errors would come up after that is fixed.  So, it's going to take a non-trivial amount of work to get this code ready for the matlab converter.  Likely can be done in-stride with the other development, but will need to be deliberate from the start.  If we have <10 auto check issues in OWENS now, but over 800 in the other optimization code, it sounds like we should definitely start going in the right direction NOW.  

Also, during my reading, it appears that matlab can compile binaries for other operating systems, but that it needs to be compiled from the target system (i.e. run the matlab compiler on a mac for a mac binary).  Another interesting thing was that the compiled matlab (according to the tutorials) isn't necessarily going to be faster than the interpreted matlab.  The c code may be faster, but may be due in part to the changes required to get matlab to spit out the code.
 
 
**Fast Input Files First Look**
Thursday, February 27, 2020
11:21 AM
 
Downloaded the example FAST input files from the docs, seems straightforward, similar to what I've used before with CCBlade.  Looking through the other files, I had the following thoughts:
 
We will need to choose a frame of reference for the blade geometry relative to the existing one: 
A) Extreme precone/curve and have to resolve the fact that the tower will be 90 degrees to the turbine without a transformation
B) Negative precone/curve and 90 degree transformation on the hubs.  
C) Use separate definitions rather than trying to use the old with modifications
 
The material properties, airfoil data, and initialization/driver files look to be straightforward to comply with for inputs to OWENS.
 
 
**Running Owens**
Monday, March 2, 2020
10:19 AM
Pulled the OffshoreVAWTmodels - Copy.zip file off of \\snl\Collaborative\windandwater\4BrandonE which appears to be a snapshot of the models.  Following the readme and running the transient driver worked, but there was an issue with wavec in plotting.  Turning off the hydro analysis and it seems to be running fine now.  Hasn't completed after a few minutes.  
 
**Write Deployable MATLAB Code**
Tuesday, March 3, 2020
3:56 PM
 
https://www.mathworks.com/help/compiler_sdk/ml_code/writing-deployable-matlab-code.html
 
We should start with only the code we want to deploy on unix: i.e. pre/post processing may be able to be postponed

 




**Compiling: Resolving Unsupported Function Calls**
Monday, March 2, 2020
10:30 AM
Adding paths: the compiler will figure it out on its own, but matlab won't, so add path if not deployed.
if (~isdeployed)
    VAWT_Toolbox_path_main = '../deps/OWENSvawtFoa/';
    addpath(VAWT_Toolbox_path_main)
end
Here are a list of supported functions: https://www.mathworks.com/help/simulink/ug/functions-and-objects-supported-for-cc-code-generation.html
 
Running the matlab code readiness tool on platformTests_transient_Mkprop, yielded the following unsupported functions.  The following are my work-arounds/new functions.
 
Fscanf - several other options in the list of supported functions, like fseek
 
Addpath - directly call out all paths, keep them in structures to be clean.
 
Str2num - use str2double
 
Cd - restructure code, see: https://www.mathworks.com/help/compiler_sdk/ml_code/writing-deployable-matlab-code.html
 
Robust way of extracting the file directory regardless of / or \:
last_delimiter = find(or(inputfile == '/', inputfile == '\'));
fdirectory = inputfile(1:last_delimiter(end));
 
 
Ismembc - use ismember
 
Warning - convert to print statement
 
Importdata - use an approved file reading method from link, may have to write small function to get the right format
 
Pwd - same as cd
 
Keyboard - restructure code: should not be hitting the keyboard in the middle of a batch parallel run on a HPC cluster
 
Strsplit - write own splitter logic, or use something like strfind in the approved list
 
Movefile - restructure so moving is not needed, or read in and rewrite the file elsewhere, and then delete the original file.
 
Getframe - This has to do with creating a video.  It is possible to restructure the code to simply output images and post process them into a video.  Also may be problematic to include image creation that uses gpu if there is no gpu on the cluster being run on, so may need to push to post processing, depends on if gpu is used etc.
 
Genpath - same as cd
 
VideoWriter - same as getframe
 
Textscan - same as fscanf
 
Movegui - same as getframe
 
Sscanf - same as fscanf
 
Clc - remove as it is matlab specific to the console.  Could put in printing levels.
 
Unix - remove since this is specific to wavec, seems to be dependent on hard coded file paths, and could easily be unified for the coupling with the new hydro tool.
 
Eigs - use one of the other eigenvalue/vector functions from the link
 
Writetable - use something like fwrite or fprintf
 
Cell2mat - restructure to not require this type of conversion
 
Annotation - move to post processing or include in legend, or remove.
 
 
All Files called by platformTests_transient_Mkprop 
 
https://www.mathworks.com/help/matlab/matlab_prog/identify-dependencies.html





 
**Successfully Compile OWENS**
Tuesday, March 3, 2020
5:17 PM
 
Put all required code in single repository and begin tracking changes
Get it running again in that environment
Create Short Unit Test to ensure code changes don't break physics (check #s based on original code)
Implement changes required as previously recorded
Attempt to create code and compile, also attempt to compile directly
Resolve any final issues associated with compilation failure
Successfully compile
Run on HPC/Unix
Resolve issues associated with running
Potentially merge changes with Brandon's changes
Map out next steps - DAKOTA and Parallelization, interfacing with other packages, cleaning up/simplifying code
 

How to preinitialize arrays of structs:

https://www.mathworks.com/help/simulink/ug/defining-arrays-of-structures-for-code-generation.html

It looks like you just have to initialize the correct type of variable, not necessarily the size, ie use zeros if you don't know the NxM etc.  

Some best practices
https://www.mathworks.com/help/simulink/ug/best-practices-for-defining-variables-for-c-c-code-generation.html

_______________________

If the type is unchanging and easily inferred (i.e. not a for loop), then you don't have to declare it.  

All variables used must be declared on all execution paths (i.e. must always have an else for each if, or be declared outside the if)

**All if statements must have an else, or the equivalent such that all --LOCAL-- execution paths possible are covered**

Structs must be build in the same order on all execution paths and cannot be used until they are finished being constructed

_____________________________

**Mar 24 2020**
4:55pm

Finally got it to create the source code, 91 commits later, each with some sort of restructuring/typsetting.  About 30% was done with the regular matlab editor suggestions, another 70% with the coder GUI in matlab, and the last 10% post compilation via mex (built into the coder GUI).  The most difficult issues to figure out were also in the 70% chunck.  Basically, you have to code as if you were coding in C++: declare your variables, and don't change their types or sizes after they are declared.  You can have variable size arrays, variable at the time of initialization.  I haven't played with it, but it appears that even that can be shut off for speed in the coder settings.  It is really frustrating to do this in matlab since whenever you make a mistake (like accidently creating a row vector instead of a column vector), it just plows through the rest of the program and resizes/retypes everything without telling you.  For this reason, it is my opinion that matlab is a terrible development tool. It incentivizes terrible coding practice, with and emphasis on matrix based syntax, opposite to any HPC code.  It is 2x slower with code optimized for matlab, then 1000x slower for code written to satisfy the coder requirements, also further incentivizing the wrong direction.  So you have to essentially write obtrusive matlab code, making up tons of sub functions to try to recapture the 60% of capability you loose in conforming with the coder requirements, then you have to generate the C++ code, and then compile it.  Why not just start with julia, retain 100% of your functionality, and have it run on HPC resources out of the box, and optimize your code as needed?  Good luck optimizing the C++ output of matlab.

This restructuring took approximately 2 weeks of work, for which I am confident I could have just translated OWENS into Julia. 

_________________

**Mar 25** 1:00pm

Speed comparison of compiled code:

- Matlab: 17s
- Matlab Mex: 34s
- Compiled C: 61s

In addition to yesterday's log, I'd say that compiling matlab is a solid loser. 92 Commits, 4 weeks, 228k Additions, and 112k Deletions to get a 3x slowdown.  If that effort were put towards translating into Julia, it would have likely taken less effort, and given a likely 2x or more speedup.  Why less effort?  Julia syntax is very similar to matlab, and julia will warn you before bulldozing through the code redefining all of your types and sizes.  For the VAWT AC code, it took me less than 8 hours to translate the code, then 2x that again to find the handful of array orientation mistakes made during the translation (since matlab 1D arrays are typically column vectors instead of row vectors, and zeros(5) makes a 5x5 instead of a straight 5 array of zeros, etc).  The owens repository I got to run C code is 12110 lines of code long. The C output is 29375 lines, 3642 for the .h and 25733 for the .c  VAWT-AC julia is 882 lines, with 1753 for the compiled root finding dylib, the matlab is 1046 lines.

Yes, the matlab code could be written better to get the c code more optimized, but wouldn't it be time better spent (and a heck of a lot easier) going stright for a HPC code like Julia.  That way we wouldn't have to go through three stages to get the code compiled.

Used the example main.c and main.h, and the following command to compile (using the packaged output of coder): gcc -Wall *.c -o pleasework.  The -Wall shows extra warnings the compiler normally ignores

**Mar 25** 5:30pm

Code review with Brandon where we agreed to

- Continue on the matlab path until we have the big pieces in place, then evaluate as to the best path for HPC
- Try the full cactus sim time and see time difference
- Try turning off the file save portion and see time difference

- Most immediate next step is coupling method
- Then Translating the modal and flutter analysis
- Then unsteady stall
- Also getting rally setup




||Matlab|Matlab MEX|Compiled C|Octave|
|---|---|---|---|---|
|Lines of Code|12,110 lines |N/A|29,375 lines|Runs .m file|
|Execution Time Short (0.1s) & File Output|21s|34s|61s|~400s|
|Execution Time Short (0.1s), No File Output|14s|19s|60s|~400s|
|Execution Time LONG (19.1s), No File Output|13m 40s(9s /100 time steps)|4m 31s (3s/100 time steps)|~3hr (2min/100 time steps)|~15hr (10min/100 time steps)|


**March 31 2020** 11:45am

Profiling the super slow C code:

brew install gperftools
brew install graphviz
brew install ghostscript

I copied the input and output folders into my coder "package" unzipped folder with the example main.cpp and main.h from the codegen folder copied. I did also switch to C++, but is seems to run in about the same time as C 

I had to disable address space layout randomization (ASLR) (https://stackoverflow.com/questions/10562280/line-number-in-google-perftools-cpu-profiler-on-macosx)

so in the end I ran the following commands

    c++ -std=c++11 -lprofiler -g -Wl,-no_pie *.cpp -o please4work

    env CPUPROFILE=/tmp/prof.out ./please4work 

    pprof --web --functions ./please4work /tmp/prof.out

(Note that you have to change the default .svg program in finder.  I changed mine to safari)

**Profiler Results**

So, it looks like there is some serious time being spent in the StructuralDynamicsTransient function, and not really the sub functions.  Looking at the code output, it looks like this is likely a memory issue where matlab is very inefficiently forcing c++ to duplicate its memory management schemes.  I'm sure a CS major could tell us the clear problem and solution in about 60 seconds.

![Profiler Results](./figs/profiler_output.png)

**April 6 2020** 12:20pm

Working on debugging the slowdown.  Also got the modal analysis to compile, but is not outputting the correct answers, possibly a complex number issue.  Ways I've tried to attack this issue

 - Compiling binary directly from matlab: Issues in getting the compiler to link to matlab.  Issue is common, but poorly documented

 - Running mex from octave: Requires some significant intrusive changes: https://hgomersall.wordpress.com/2013/02/19/squishing-matlab-mex-files-into-octave/
 - Running mex from python: There is a matlab API for this
 - Creating mex file from octave: spent about 15 min on this and it did compile, but will not run.
 
However... The mex function calls the matlab libraries, so the mex itself doesn't contain all the code.  Therefore, the option would be to compile the matlab code (possibly with the mex as part of it), but that requires the matlab runtime.
 
Now on to the other ways I've tried to attack this issue:
 
 - C++ debugger: gdb.  Some non-trivial issues getting it to run - keychain permissions, etc.  Turns out none of them work and you just have to run it as sudo.
 
**April 6 2020** 3:50pm

Ok, using the gdb and the simple debugging principle that if the is one bottle neck taking a large amount of time, then there is a high probability randomly debugging and stopping the debugging will land you at the trouble spot.  Doing the process several dozen times, yielded the following, all in structuralDynamicsTransient.cpp (keep in mind, it could have landed anywhere in the program):


|C++ Line|# Hits|
|---|---|
|473|15|
|429|5|
|428|3|
|471|1|
|427|1|
|472|1|

They all map to a **SINGLE** matlab line with basic matrix multiplication of fixed size matrices (for a given program run):
structuralDynamicsTransient.m:339

```matlab
Kg = transMatrix'Kg*transMatrix;
```

why on earth it spits out the following c++ code is a mystery yet to be solved

```c++
  i = a->size[0] * a->size[1];
  a->size[0] = model_jointTransform->size[1];
  a->size[1] = model_jointTransform->size[0];
  emxEnsureCapacity_real_T(a, i);
  loop_ub = model_jointTransform->size[0];
  for (i = 0; i < loop_ub; i++) {
    m = model_jointTransform->size[1];
    for (boffset = 0; boffset < m; boffset++) {
      a->data[boffset + a->size[0] * i] = model_jointTransform->data[i +
        model_jointTransform->size[0] * boffset];
    }
  }

  if ((a->size[1] == 1) || (Kg->size[0] == 1)) {
    i = y->size[0] * y->size[1];
    y->size[0] = a->size[0];
    y->size[1] = Kg->size[1];
    emxEnsureCapacity_real_T(y, i);
    loop_ub = a->size[0];
    for (i = 0; i < loop_ub; i++) {
      m = Kg->size[1];
      for (boffset = 0; boffset < m; boffset++) {
        y->data[i + y->size[0] * boffset] = 0.0;
        inner = a->size[1];
        for (coffset = 0; coffset < inner; coffset++) {
          y->data[i + y->size[0] * boffset] += a->data[i + a->size[0] * coffset]
            * Kg->data[coffset + Kg->size[0] * boffset];
        }
      }
    }
  } else {
    m = a->size[0];
    inner = a->size[1];
    loop_ub = Kg->size[1];
    i = y->size[0] * y->size[1];
    y->size[0] = a->size[0];
    y->size[1] = Kg->size[1];
    emxEnsureCapacity_real_T(y, i);
    for (j = 0; j < loop_ub; j++) {
      coffset = j * m;
      boffset = j * inner;
      for (b_i = 0; b_i < m; b_i++) {
        y->data[coffset + b_i] = 0.0;
      }

      for (k = 0; k < inner; k++) {
        aoffset = k * m;
        totalNumDOF = Kg->data[boffset + k];
        for (b_i = 0; b_i < m; b_i++) {
          i = coffset + b_i;
          y->data[i] += totalNumDOF * a->data[aoffset + b_i];
        }
      }
    }
  }

  if ((y->size[1] == 1) || (model_jointTransform->size[0] == 1)) {
    i = Kg->size[0] * Kg->size[1];
    Kg->size[0] = y->size[0];
    Kg->size[1] = model_jointTransform->size[1];
    emxEnsureCapacity_real_T(Kg, i);
    loop_ub = y->size[0];
    for (i = 0; i < loop_ub; i++) {
      m = model_jointTransform->size[1];
      for (boffset = 0; boffset < m; boffset++) {
        Kg->data[i + Kg->size[0] * boffset] = 0.0;
        inner = y->size[1];
        for (coffset = 0; coffset < inner; coffset++) {
          Kg->data[i + Kg->size[0] * boffset] += y->data[i + y->size[0] *
            coffset] * model_jointTransform->data[coffset +
            model_jointTransform->size[0] * boffset];
        }
      }
    }
  } else {
    m = y->size[0];
    inner = y->size[1];
    loop_ub = model_jointTransform->size[1];
    i = Kg->size[0] * Kg->size[1];
    Kg->size[0] = y->size[0];
    Kg->size[1] = model_jointTransform->size[1];
    emxEnsureCapacity_real_T(Kg, i);
    for (j = 0; j < loop_ub; j++) {
      coffset = j * m;
      boffset = j * inner;
      for (b_i = 0; b_i < m; b_i++) {
        Kg->data[coffset + b_i] = 0.0;
      }

      for (k = 0; k < inner; k++) {
        aoffset = k * m;
        totalNumDOF = model_jointTransform->data[boffset + k];
        for (b_i = 0; b_i < m; b_i++) {
          i = coffset + b_i;
          Kg->data[i] += totalNumDOF * y->data[aoffset + b_i];
        }
      }
    }
  }
```

Trying the following shaves 2seconds/ 100 time steps off.  Perhaps it will give a better c++ output

```matlab
% Kg = transMatrix'*(Kg*transMatrix);
Kg = full(sparse(transMatrix')*sparse(Kg)*sparse(transMatrix));
```

Much much better! 24X Faster!  Matlab tests pass and c++ test passes for transient!

||Matlab|Matlab MEX|Compiled C++|Octave|
|---|---|---|---|---|
|Execution Time Short (0.1s) & File Output|21s|NA|5s|many minutes|
|Per 100 Steps|7s|2s|5s|many minutes|


This is the output code, (that one liner was in its own function to start with, and now matlab finally puts this in its own function) and the code is significantly simpler.  Also, as a note, it didn't seem to have any effect when I tried to initialize variables outside of a for loop, it seemed to just do as it pleased.

```c++


static void applyConstraints(emxArray_real_T *Kg, const emxArray_real_T
  *transMatrix)
{
  emxArray_real_T *b_transMatrix;
  int i;
  int loop_ub;
  emxArray_real_T *this_d;
  int cend;
  emxArray_int32_T *this_colidx;
  int t3_m;
  emxArray_int32_T *this_rowidx;
  emxArray_real_T *t2_d;
  emxArray_int32_T *t2_colidx;
  emxArray_int32_T *t2_rowidx;
  emxArray_real_T *t3_d;
  emxArray_int32_T *t3_colidx;
  emxArray_int32_T *t3_rowidx;
  int this_m;
  int this_n;
  emxInit_real_T(&b_transMatrix, 2);
  i = b_transMatrix->size[0] * b_transMatrix->size[1];
  b_transMatrix->size[0] = transMatrix->size[1];
  b_transMatrix->size[1] = transMatrix->size[0];
  emxEnsureCapacity_real_T(b_transMatrix, i);
  loop_ub = transMatrix->size[0];
  for (i = 0; i < loop_ub; i++) {
    cend = transMatrix->size[1];
    for (t3_m = 0; t3_m < cend; t3_m++) {
      b_transMatrix->data[t3_m + b_transMatrix->size[0] * i] = transMatrix->
        data[i + transMatrix->size[0] * t3_m];
    }
  }

  emxInit_real_T(&this_d, 1);
  emxInit_int32_T(&this_colidx, 1);
  emxInit_int32_T(&this_rowidx, 1);
  emxInit_real_T(&t2_d, 1);
  emxInit_int32_T(&t2_colidx, 1);
  emxInit_int32_T(&t2_rowidx, 1);
  emxInit_real_T(&t3_d, 1);
  emxInit_int32_T(&t3_colidx, 1);
  emxInit_int32_T(&t3_rowidx, 1);
  b_sparse(b_transMatrix, t2_d, t2_colidx, t2_rowidx, &cend, &loop_ub);
  b_sparse(Kg, this_d, this_colidx, this_rowidx, &this_m, &this_n);
  d_sparse_mtimes(t2_d, t2_colidx, t2_rowidx, cend, this_d, this_colidx,
                  this_rowidx, this_n, t3_d, t3_colidx, t3_rowidx, &t3_m,
                  &loop_ub);
  b_sparse(transMatrix, t2_d, t2_colidx, t2_rowidx, &cend, &loop_ub);
  d_sparse_mtimes(t3_d, t3_colidx, t3_rowidx, t3_m, t2_d, t2_colidx, t2_rowidx,
                  loop_ub, this_d, this_colidx, this_rowidx, &this_m, &this_n);
  i = Kg->size[0] * Kg->size[1];
  Kg->size[0] = this_m;
  Kg->size[1] = this_n;
  emxEnsureCapacity_real_T(Kg, i);
  emxFree_real_T(&b_transMatrix);
  emxFree_int32_T(&t3_rowidx);
  emxFree_int32_T(&t3_colidx);
  emxFree_real_T(&t3_d);
  emxFree_int32_T(&t2_rowidx);
  emxFree_int32_T(&t2_colidx);
  emxFree_real_T(&t2_d);
  for (i = 0; i < this_n; i++) {
    for (t3_m = 0; t3_m < this_m; t3_m++) {
      Kg->data[t3_m + Kg->size[0] * i] = 0.0;
    }
  }

  for (loop_ub = 0; loop_ub < this_n; loop_ub++) {
    cend = this_colidx->data[loop_ub + 1] - 1;
    i = this_colidx->data[loop_ub];
    for (t3_m = i; t3_m <= cend; t3_m++) {
      Kg->data[(this_rowidx->data[t3_m - 1] + Kg->size[0] * loop_ub) - 1] =
        this_d->data[t3_m - 1];
    }
  }

  emxFree_int32_T(&this_rowidx);
  emxFree_int32_T(&this_colidx);
  emxFree_real_T(&this_d);
}

```

**April 6 2020** 5:40pm

Moving on to the modal analysis verification issue:

||Matlab|Matlab MEX|Compiled C++|Octave|
|---|---|---|---|---|
|Correct Output|yes|yes|no-visually is way off|yes-visually, appears octave writes -0.0 instead of 0.0|
|Execution Time|11s|not recorded|80s|~5min|

So, we do have **a** way to run it on the compute nodes via octave.  It's slow, but does work.

Ok, let's try the debugger route.

compile the code using the -g flag to include the debugger ability
- c++ -std=c++11 -g *.cpp -o pleaseMwork

now run the code in the directory it was compiled from (i.e. copy the necessary run folders into the build directory) (I installed gdb via brew) (cheat sheet for gdb: https://darkdust.net/files/GDB%20Cheat%20Sheet.pdf)
- sudo gdb ./pleaseMwork

Place breakpoints and run (the first is so the program will start debugging right off)
- break main.cpp:61
- break extractFreqDamp.cpp:182
- run



**April 7 2020** 8:30am

It catches at a break point and then we can continue (c) or quit by (cntrl-c). (info locals) shows all of the local variables.  (print variablename) prints the variable name.  Keep in mind, c++ matrices are row arrays that loop around, so to get the second row, column one of a 414x414 array you'd access 414 (remember zero indexing), and the command would likely be "print variable->data[414]".  

I've identified that its the damping matrix, C, that has all the values negated.  I've traced it backwards to the timoshenkoelement 

**April 7 2020** 3:30pm

it came down to this line of code calcTimoNL.m at about line 580:

```matlab
[Ce] = mapMatrixNonSym([zm,C12,C13,C14,zm,zm;
     -C12',zm,C23,C24,C25,C26;
     -C13',-C23',C33,C34,C35,C36;
     -C14',-C24',C43,C44,C45,C46;
     zm,-C25',-C35',-C45',zm,zm;
     zm,-C26',-C36',-C46',zm,zm]);

```
All of the inputs had the correct signs, but the matrix it created did not.  I explicitely built the matrix element by element and it changed the C++ output from a very odd mapping scheme to nearly following what I implemented (which was much simpler and easier to read).  Still isn't quite right, but at least now the mode shape numbers are just negated, but appear to have the correct magnitude of values.  Will debug the C,K,M matrices again.

4:00pm

Ok, it looks like there is a bug in the translator, it is mixing up the rows.  While true that row 3 index (3,1) (i.e linear index 24 is set to the negative C12 value accessed, the matlab code is assigning it to the wrong index, like it is getting confused trying to write more "efficient" code, which is only ending up creating terrible non-working code.

```c++
  //  Row 1
  ktemp2_tmp = elStorage->C12[0] * Oel[2];
  ktemp2[24] = ktemp2_tmp;
    .
    .
    .
  //  Row 3
  ktemp2[2] = -ktemp2_tmp;
```

Now that I have had a first in-depth scrape with matlab's C++ code output, I'd give it a D-.  In other words, good effort, but the product is scrappy at best for a HPC quality code. It is difficult to follow (without the master Matlab script side by side it would be impossible), it has redudnant code everywhere, including two and three copies of the same functions all side by side (in a 500 line matlab script, that means that each function is ~1500 lines for a total of about 5000 lines, and the other functions aren't called), and finally, it fails at what it is supposed to be best at.  Matlab stands for matrix laboratory, but when it spits out a complex, non-standard matrix multiply function at random that is 24-50x slower than the standard, while using the standard fast method elsewhere.

4:40pm

It appears that the translator makes the same assumption I would when creating a matrix, that to access the 3,1 element of a 12x12 matrix, one would specify 24, assuming row folding.  However that is not the case.  It uses column folding, so element 24 accesses 1,3, or the transpose.  Since the matrix at hand is negative symmetric, we had the masked issue.

5:40pm

Both are using row major and both match, its just that the debugger makes it look column major, and uses the array syntax that matlab would use for column major (i.e. commas).  So matlab didn't get this part wrong, I misunderstood the debugger's/c++, but the rest of the auto generated code is still scrappy.  This means I need to backtrack to the function that outputs the mode shapes... :(

As a note, this process is quite tedious:

Change matlab code -> Generate c++ code (~3min) -> setup folder structure (30s) -> compile c++ (30s) -> get debugger running (2min) -> step to desired location and print (1min).  That's 7 minutes of overhead for each code change.  Using Julia it would be about 20 seconds, and matlab it's like 10 seconds thanks to the multimillion dollar debugger.

Ok, we know that the mode shapes are off, let's check the Mg, Cg, Kg matrices.  
- A half dozen samples of the M matrix all over show equivalence (M: linearAnalysis:145, C: linearAnalysis:415
- Same story for the K matrix (similar location)
- Same story for C matrix (similar location)

**April 8 2020** 8:35am

- Mg_Total good
- Cg_Total good
- Kg_Total good

Dive into eigsolve again.

- sysMat appears to be the same

4:30pm

Looking into all of the different ways to solve the eigenvalue problem: eigs(A,B,nmodes,sortingmethod), polyeig(M,C,K), eig(A), eig(A,B), etc.  It appears that while the eigs implementation in the code appeared to give different results, it is now appearing that that was only since it was sorted from smallest to largest (absolute, including imaginary number).

However, using the sorted eig output, the mode shapes are still different from the sorted eigs output, even when using eigs to solve for all of the modes.  It does appear however that the worst residual of the eig output is much lower than eigs (e-07 as opposed to e-02).  Therefore it may be that the mode shapes from eig are more accurate than eigs???  This isn't the case for the dominant modes though, since the eigs residual is much lower (eigs 1e-16 and eig 1e-10).  So the differing methods don't match, and then when we compile the code and run it, there is some similarity in the numbers, but some of them are still way off.  So, I vote that since I've spent over 2 days on this single particular problem, and it appears to be rooted in the eigen solve, I move on and leave these thoughts for when we come back to it.  However, my official opinion is that we should just switch to a more scientific computing friendly environment, and ditch trying the get the Grinch to do what we want with a 32.5 foot pole.

    1) Need to develop a real validation case
    2) If none of the eig solving techniques work in C++ (i.e. we can't use them for our purposes), we decide to either
        a) Just use octave and don't worry about the 5 min run time
        b) Develop our own eigensolve algorithm that plays nicely with the C++ translator



**April 9 2020** 11:59am

Moved on to the flutter analysis.  It looks like the manual flutter is already implemented alongside the modal analysis.  Working on the automated flutter analysis.  For a coarse number of modes and RPM levels it is having a terrible time converging.  I am going to try a root solver instead of damped iteration.  

Switched to a 1D minimizer since it doesn't appear that some mode/RPM levels have solutions.  Another observation is that this auto flutter method is showing that the underlying analysis, physically accurate or not, can be quite discontinuous at the flutter solutions.

**April 13 2020** 4:30pm 

Compiled and successfully ran the automated flutter analysis.  No verification yet since that depends on the eigensolve.

Found out about and created a space on confluence.sandia.gov.  They have an integrated instance of lucidchart!  I don't know if that includes the realtime collaboration in the lucidchart instance itself, but it's great!  Also appears to have general collaborative documents -> i.e. could be a replacement for google docs.

Using the sandia instance of lucidchart, I created the following flowcharts. (Can also export as .vdx for legacy compatability)

![Internal Code Structure](./figs/OWENS.pdf)
![High Level IO Draft](./figs/ARCUS_IO.pdf)


6:15 pm

Plotting the modes shows only minor discrepancies (discounting inconsistencies between phases and directions) between the matlab and C++, except for mode 5, which one of the phases doesn't match at all. Left to right:

EIGS Output ...............................................................  EIG Output ...............................................................  C++ Output (Based on EIG code)

![Mode 5](./figs/mode5.png)

**April 14 2020** 7:00pm

Today I learned that eigenvector magnitude is meaningless, that the vector magnitude is essentially a function of good numerical methods.  Therefore, the unit test needs to be updated to compare the normalized mode shapes.  Now as far as switching up the phases and directions, the back of my mind is telling me that is not a problem also because of the numerical methods, mostly for higher order modes.  I also learned that the polyeigs output versus the eigs square output are the same modeshapes, it is just that the square matrix with the repeated diag and zeros output repeats the modes.  It may be faster to use the polyeigs function due to this.  Will need to look into that tomorrow after updating the unit tests, and also putting the results of the hand test here.

**April 15 2020** 10:55am

Figures as promised.  I also updated the modal verification method as planned and it all passes, even mode 5 which is very interesting, the scaling of the C++ of mode 5 must just have been very different than the others.  Would be interesting to plot the mode shapes with equal scaling, and also verify that the code is correctly using them for displacements.

Non Unitized, with the second/repeated set of eigenvectors included (first mode from the by hand script)

X-axis is node number, y-axis is nondimensionalized magnitude
![non_unitized_eigvec](./figs/non_unitized_eigvec.jpg)

When unitized, they are all exactly the same shape.
![unitized_eigvec](./figs/unitized_eigvec.jpg)

**April 16 2020** 10:15am

Timed results of differing eig methods, times are for the entire run:

|Method|Time|
|---|---|
|Polyeig|93s|
|Split out M EIG|94s|
|Single Matrix EIG|57s|
|Matlab single mat EIG|10s|

Also ran the profiler on the single matrix eig and the issue is within xzggev.cpp, which appears to be the eigensolve itself.  The function is basically illegible, so it seems that our 57 seconds is as good as we get without CS help and migrating the code permanently out of matlab.  It is potentially possible to play with the types/sizes (making them more concrete) going into the matlab solve to see if the output function can be simplified.

Hard coding the matrix size going into the eig solve gives a run time of 38 seconds (single matrix eig), so definitely some potential there.

**July 13 2020**

Switching to julia translation.  Aero module is put together, but some bugs in the dynamic stall and also in the RPI fundamental effects that I want to hold off on until my mind clears in a week (since I don't have a coding buddy).

Starting translation log: so that we know just how challenging and resource intensive it is to translate:

10:20am

file extension, comments, looked for auto translator, but don't think that it will work, plus it isn't outside eyes free...

' to "

fprintf to println

access elements ( to \[

";" not needed unless multiline-line

platformprop struct to declared mutable struct at top of file, should be changed to regular struct for speed and safety.

test_owens done: 10:50am, moving on to owens

include owens in previous file (should be later better organized)

change function header return to a return at the end of the file.

;

'

change varargin to the optional julia arguments with defaults

model mutable struct

fopen to open

translating find(or logic 11:20

11:45 switch 1:15pm, got it at 1:35

fgetl to readline

[

string2double to parse(Float64, string)

[stringa stringb] to string(stringa, stringb)

find to findall and associated logic done at 2:15
Multitasking during meeting and CACTUS questions
translating short readmesh function 3:50
outputs not in []
translating readBladeData
max to maximum for arrays
~ to ! for not
~ to _ for unused arguments/outputs

Translating readBCdata.jl done at 6:00

Translating readElementData 
getting array of structs Out at 6:30 80 lines/hr average

**July 14 2020**

Continuing readElementData - \[ and file reading done 9:30, verifying outputs to current point - minor error in previous code with filname concatenation, otherwise looks good from a sample. 9:50

Starting Read Joint Data done 10:05, checking, looks great

Starting read nodal terms, done 10:40

Starting GenerateOutputFilename, done 11:00

Internet installed at new house and lunch and transferred workstation 116 lines/hr average

2:35 Starting GyricFEA.createJointTransform 
Large function, also calculateLambda and calculatelambdaslim
watch out: matlab zeros(12) is a 12x12 whereas julia follows a more logical output of 12

eye(3) to 1.0*LinearAlgebra.I(3)

inv(A)*b to A\\b (for efficiency) Done 5:30, 180 lines/hr average

Starting calculateBCMap, done 5:45

Starting constructReducedDispVectorMap, 
find(ismember(dofList,i)) to findall(x->x==i,dofList)[1] Finished 6:00pm

**July 15 2020**

Starting readNLParamsFile at 7:15
strcomp to occursin - found it redundant and not even reading in the file, starting modalExec at 7:30, 7:45 looked at code count and it should take me about 12 hours, looking at the transient execution, it looks to be only about twice that, so I think I should focus on that aspect since that is the glue code that we're initially interested in. 

8:30 owens.jl parts that are for transient,10:00 Pi Meet, 11:00-11:30 done, Lunch, 12:00 continue into transient exec



**August 3 2020**

Translated file input and up to transientExec as well as a few small functions.  Average translation speed is around 100 lines/hr

As an incremental approach, I have Julia call Matlab directly for the functions that I'm not going to translate at the moment, or which I want a smaller stepping stone.  Then I create a dynamic library from the specific function.

I did a simple test function that just takes in data of various types, prints them out and returns them.  It seems that the ccall function for julia calling c/fortran functions doesn't play well with structures, so I broke everything apart, and then reassembled it on the other side for structuralDynamicsTransient.  

On my simple test function, here are the relative speed results (averaged over 1000 function calls)

|Method|Time|
|---|---|
|Average Time Elapsed Pure Matlab|0.00058s|
|Average Time Elapsed Julia Calls Matlab|0.041015s|
|Average Time Elapsed Julia CCALL|0.0000765s|
|Average Time Elapsed Pure Julia|0.00017183s|

|Method|Factor|
|---|---|
|(Pure Matlab)/(Pure Julia): Julia is | 3.37 x faster |
|(Julia Calls Matlab)/(Pure Julia): Julia is | 238.70 x faster |
|(Pure Julia)/(CCALL): Julia is | 2.24 x SLOWER |


Now since the structuralDynamicsTransient transient function is much more complex, we'll have to work through all of type issues.

Change matlab code to include print statements, create the dynamic library, run it in Julia, overhead of about 2mins debugging an issue that wouldn't exist in pure julia.  After going through all of this process, I think that I will update my estimated level of effort of matlab->c dylib called by julia and just translating the matlab to julia to be half the effort, i.e.

So, while debugging, I'm trying to evaluate the size of an array in the matlab generated c code during the Julia runtime.  In matlab I put in code and print statements to show the size of the array in question.  The matlab coder function hard codes the original results and doesn't do the calculation I specified...

I just found an oddity - there seems to be a bad area of data input - i.e. for the 106 and 107th inputs to ccall, they cause a sigfault if called/accessed, but if I put two bogus inputs so it pushes my good inputs down, then it runs fine and the good inputs can be accessed. Very odd...

**Aug 4 2020**

Rearranged inputs per type, is now giving me an read-only memory error instead of a sigfault which is nice, tried the output flattening, but still had the same issues.  

Now several hours later, it appears that some memory items aren't making it over... if I rearrange the output I get different areas where it sigfaults.  Printing out the data shows this to be the case.  I found a similar issue on stack overflow or the like that was posted 8 days ago and hasn't been answered...  I may have to come to the conclusion that ccall can't handle tons of separated io.

**Aug 5 2020**

Hard coded all of the inputs, and that ran fine (outputs passed by reference), next de-hard-coded in all of the K11 type data and that ran fine (so the only input type is Ptr{Float64} which is type for an array, next de-hard-coded almost all arrays and that ran fine, then all of the Float64s and it ran fine, and then the rest of the arrays and it runs fine, so all we're left with are booleans.  Regular bools work fine, the array of bools causes the weird segfault with seemingly unrelated data.  Changing the code to avoid an array of bools works fine.  Now running the verification script, there were nans in the output I hadn't seen before.  Turns out the auto unsafe convert from an array of ints to an array of floats doesn't work, so doing it manually now it works great and the test passes.  This was extremely difficult and tedious to debug considering the matlab translation overhead...  In all it cost me about 2.5 days, which could have put me with a full translation of the same function, so now between getting it to run in C and then getting the interface, I'm a factor of 2 in favor of pure julia.  We'll see what the speed difference is...

Time Comparison for Calling StructuralDynamicsTransient Function:

|Method|Appx Ave Time|Relative Speedup against Original Pure Matlab|
|---|---|---|
|Pure Matlab Original|0.08 s|NA|
|Pure Matlab Variable Breakout|0.15 s| 2x slower|
|Julia Calls Matlab Variable Breakout|0.75s|10x slower|
|Julia Calls Matlab Original|0.25s|3.1x slower|
|Julia Ccall Variable Breakout|0.035s|2.3x FASTER|
|Pure Julia No Breakout|???|???|

That's 2x faster compared to the original including all of the variable breakout, etc.  (That may not have much effect since Julia is pass by reference everywhere unless explicitely stated whereas matlab is copy always).  Matlab without the breakout is much faster.

Note: Don't forget to kill/refresh Julia or else it will won't reload the dylib.  Same for any precompiled library, module, etc.

Takeaways: 

1) You must pass in all of the expected types, not relying on auto convert to save your bacon

2) All of the issues were related to type definitions, so debugging time is best spent verifying data actually made it into the function correctly, i.e. forward debugging as opposed to reverse debugging.

3) Input types should be standard/simple - structs don't seem to work easily, neither nonstandard things like bitarrays.  I.e. multidimensional arrays of large size are fine, as well as having a lot of them.



**Oct 2 2020**

Aero module VAWTAero is looking great, got the 3D effects and RPI method pretty well worked out (all documented in my paper in the same repo), and now I'm back to the coupling.  I took a building block approach and used the matlab version of AC and structure as I had it before.  Used the julia matlab direct interface, got me up and running in no time, and I was even surprised when looking at the unit test outputs that the reaction forces (which were the only part of the analysis changed by swapping out the aero) were remarkably similar.  I.e. if it was 7.325e7 originally, mine was something like 7.864e7 and likewise for large, small, positive, and negative values.  It took a really long time to run and I learned by printing out the convergence norms below that it was because it wasn't converging to the desired tolerance, nor was the convergence changing as shown below.  

You have to view these in edit mode for the formatting to be aligned.

Running Transient
EIyz, rhoIyz deactivated
NODAL FILE NOT LOADED, CODE UNFINISHED IN readNodalTerms
INITIAL CONDITIONS NOT FULLY ENABLED
GENERATOR NOT FULLY ENABLED
Running in specified rotor speed mode
Iteration: 1
1   uNorm: 1.0e6    platNorm: 1.0e6    aziNorm: 1.0e6    gbNorm: 1.0e6
2   uNorm: 1.0    platNorm: 0.0    aziNorm: 1.0    gbNorm: 0.0
3   uNorm: 0.004307526817131652    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
4   uNorm: 4.421135958469738e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
5   uNorm: 4.605420227206929e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
6   uNorm: 4.602184904251755e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
7   uNorm: 4.611837383348467e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
8   uNorm: 4.611546630759827e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
9   uNorm: 4.605835112064324e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
10   uNorm: 4.5991908681321e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
11   uNorm: 4.595355539097174e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
12   uNorm: 4.6225230491663486e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
13   uNorm: 4.5837486107014476e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
14   uNorm: 4.61705680786041e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
15   uNorm: 4.611903727497344e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
16   uNorm: 4.5952740241501823e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
17   uNorm: 4.6086724387594544e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
18   uNorm: 4.60139942787114e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
19   uNorm: 4.6317589919520276e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
20   uNorm: 4.577669694831621e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
21   uNorm: 4.632204375052505e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
22   uNorm: 4.6024723446335977e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
23   uNorm: 4.605245827175897e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
24   uNorm: 4.615511068436272e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
25   uNorm: 4.589470479075624e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
26   uNorm: 4.6210394978822373e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
27   uNorm: 4.5916365461932027e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
28   uNorm: 4.6118391578642336e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
29   uNorm: 4.592508891321947e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
30   uNorm: 4.627477708177282e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
31   uNorm: 4.601696131322721e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
32   uNorm: 4.6188837675438925e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
33   uNorm: 4.595810241763739e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
34   uNorm: 4.603747216176561e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
35   uNorm: 4.609270538432337e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
36   uNorm: 4.603403032784982e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
37   uNorm: 4.6031321793686587e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
38   uNorm: 4.6033502795326567e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
39   uNorm: 4.621111867506377e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
40   uNorm: 4.5970091449738987e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
41   uNorm: 4.613862893391694e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
42   uNorm: 4.6110085769822753e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
43   uNorm: 4.5988174248100606e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
44   uNorm: 4.6101820029685775e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
45   uNorm: 4.6201699376668695e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
46   uNorm: 4.603181302970161e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
47   uNorm: 4.614078161454973e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
48   uNorm: 4.6070649869117375e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
49   uNorm: 4.607792805010061e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
Iteration: 2
1   uNorm: 1.0e6    platNorm: 1.0e6    aziNorm: 1.0e6    gbNorm: 1.0e6
2   uNorm: 0.7974679015578225    platNorm: 0.0    aziNorm: 0.5    gbNorm: 0.0
3   uNorm: 3.4502350407242614e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
4   uNorm: 5.25898756964089e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
5   uNorm: 5.265845528018965e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
6   uNorm: 5.265244086774431e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
7   uNorm: 5.266461714607588e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
8   uNorm: 5.263590790976053e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
9   uNorm: 5.263246933952278e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
10   uNorm: 5.265804265429286e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
11   uNorm: 5.268586137741934e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
12   uNorm: 5.262604832321805e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
13   uNorm: 5.270053320418093e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
14   uNorm: 5.268906754562389e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
15   uNorm: 5.266840122685167e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0
16   uNorm: 5.268732669916631e-7    platNorm: 0.0    aziNorm: 0.0    gbNorm: 0.0

This is basically telling me that 1) this open loop convergence isn't the best, and that a real root solver is needed and 2) For the mean time, I'll just loosen the criteria.

This is the output CACTUS vs matlab version of AC when I have the two way coupling on:

FReactionHist 2 : 1 OLD: -7.142141e+02 NEW: -3.293553e+02
       FReactionHist 2 : 2 OLD: -1.076061e+02 NEW: -2.985056e+02
       FReactionHist 2 : 3 OLD: 1.515958e+05 NEW: 1.506536e+05
       FReactionHist 2 : 4 OLD: 7.069546e+01 NEW: -7.408215e+02
       FReactionHist 2 : 5 OLD: -7.420892e+02 NEW: -6.213880e+01
       FReactionHist 2 : 6 OLD: 1.359528e+02 NEW: 1.863839e+02
       FReactionHist 3 : 1 OLD: -3.359312e+03 NEW: -1.021055e+03
       FReactionHist 3 : 2 OLD: -4.588144e+02 NEW: -1.670588e+02
       FReactionHist 3 : 3 OLD: 4.326783e+05 NEW: 4.264231e+05
       FReactionHist 3 : 4 OLD: -1.020519e+03 NEW: -8.938596e+03
       FReactionHist 3 : 5 OLD: 3.280347e+03 NEW: 4.734433e+03
       FReactionHist 3 : 6 OLD: 1.258519e+03 NEW: 1.440247e+03
       FReactionHist 4 : 1 OLD: -3.979357e+03 NEW: 1.438170e+03
       FReactionHist 4 : 2 OLD: -1.340640e+02 NEW: 3.528587e+03
       FReactionHist 4 : 3 OLD: 7.319887e+05 NEW: 7.162688e+05
       FReactionHist 4 : 4 OLD: -6.307394e+03 NEW: -2.884208e+04
       FReactionHist 4 : 5 OLD: 2.314380e+04 NEW: 2.058354e+04
       FReactionHist 4 : 6 OLD: 3.744007e+03 NEW: 2.469842e+03
       FReactionHist 5 : 1 OLD: -1.750950e+03 NEW: 4.991272e+03
       FReactionHist 5 : 2 OLD: 1.599643e+03 NEW: 7.504134e+03
       FReactionHist 5 : 3 OLD: 9.992354e+05 NEW: 9.775498e+05
       FReactionHist 5 : 4 OLD: -1.744892e+04 NEW: -5.062038e+04
       FReactionHist 5 : 5 OLD: 6.147387e+04 NEW: 3.932349e+04
       FReactionHist 5 : 6 OLD: 3.374528e+03 NEW: -3.505208e+03
       FReactionHist 6 : 1 OLD: 2.905970e+03 NEW: 5.156097e+03
       FReactionHist 6 : 2 OLD: 4.352465e+03 NEW: 8.240028e+03
       FReactionHist 6 : 3 OLD: 1.273110e+06 NEW: 1.248294e+06
       FReactionHist 6 : 4 OLD: -3.212057e+04 NEW: -6.551391e+04
       FReactionHist 6 : 5 OLD: 1.179283e+05 NEW: 5.385388e+04
       FReactionHist 6 : 6 OLD: -2.245956e+03 NEW: -1.167972e+04
       FReactionHist 7 : 1 OLD: 9.451359e+03 NEW: 5.400977e+03
       FReactionHist 7 : 2 OLD: 6.298100e+03 NEW: 9.056949e+03
       FReactionHist 7 : 3 OLD: 1.535036e+06 NEW: 1.505660e+06
       FReactionHist 7 : 4 OLD: -4.252528e+04 NEW: -7.576789e+04
       FReactionHist 7 : 5 OLD: 1.766629e+05 NEW: 6.439200e+04
       FReactionHist 7 : 6 OLD: -9.761377e+03 NEW: -1.420002e+04
       FReactionHist 8 : 1 OLD: 7.140729e+03 NEW: 5.980927e+03
       FReactionHist 8 : 2 OLD: 5.190312e+03 NEW: 9.404837e+03
       FReactionHist 8 : 3 OLD: 1.800449e+06 NEW: 1.762504e+06
       FReactionHist 8 : 4 OLD: -4.293471e+04 NEW: -7.967494e+04
       FReactionHist 8 : 5 OLD: 2.083231e+05 NEW: 7.335628e+04
       FReactionHist 8 : 6 OLD: -1.689814e+04 NEW: -1.682156e+04
       FReactionHist 9 : 1 OLD: -5.777089e+03 NEW: 5.320279e+03
       FReactionHist 9 : 2 OLD: 3.081757e+03 NEW: 7.379453e+03
       FReactionHist 9 : 3 OLD: 2.059662e+06 NEW: 2.007917e+06
       FReactionHist 9 : 4 OLD: -4.110436e+04 NEW: -7.762030e+04
       FReactionHist 9 : 5 OLD: 2.195080e+05 NEW: 8.081069e+04
       FReactionHist 9 : 6 OLD: -2.325660e+04 NEW: -2.133149e+04
       FReactionHist 10 : 1 OLD: -1.532398e+04 NEW: 4.445776e+03
       FReactionHist 10 : 2 OLD: 4.241030e+03 NEW: 5.410116e+03
       FReactionHist 10 : 3 OLD: 2.292043e+06 NEW: 2.230613e+06
       FReactionHist 10 : 4 OLD: -4.820738e+04 NEW: -7.606674e+04
       FReactionHist 10 : 5 OLD: 2.564952e+05 NEW: 8.466375e+04
       FReactionHist 10 : 6 OLD: -2.816460e+04 NEW: -2.596768e+04
       FReactionHist 11 : 1 OLD: -8.625607e+03 NEW: 3.714418e+03
       FReactionHist 11 : 2 OLD: 7.871172e+03 NEW: 5.907490e+03
       FReactionHist 11 : 3 OLD: 2.496174e+06 NEW: 2.435182e+06
       FReactionHist 11 : 4 OLD: -6.118161e+04 NEW: -7.831353e+04
       FReactionHist 11 : 5 OLD: 3.693550e+05 NEW: 9.201350e+04
       FReactionHist 11 : 6 OLD: -3.092693e+04 NEW: -3.111981e+04
       FReactionHist 12 : 1 OLD: 1.378745e+04 NEW: 3.546436e+03
       FReactionHist 12 : 2 OLD: 8.676335e+03 NEW: 5.867005e+03
       FReactionHist 12 : 3 OLD: 2.661933e+06 NEW: 2.611730e+06
       FReactionHist 12 : 4 OLD: -6.678975e+04 NEW: -8.550302e+04
       FReactionHist 12 : 5 OLD: 5.642306e+05 NEW: 1.073609e+05
       FReactionHist 12 : 6 OLD: -3.419050e+04 NEW: -3.776716e+04
       FReactionHist 13 : 1 OLD: 4.188317e+04 NEW: 5.987714e+03
       FReactionHist 13 : 2 OLD: 5.949264e+03 NEW: 6.336773e+03
       FReactionHist 13 : 3 OLD: 2.809592e+06 NEW: 2.780997e+06
       FReactionHist 13 : 4 OLD: -6.122785e+04 NEW: -9.468650e+04
       FReactionHist 13 : 5 OLD: 7.879436e+05 NEW: 1.366017e+05
       FReactionHist 13 : 6 OLD: -4.106554e+04 NEW: -4.771901e+04
       FReactionHist 14 : 1 OLD: 5.831440e+04 NEW: 9.751300e+03
       FReactionHist 14 : 2 OLD: 3.728332e+03 NEW: 8.049631e+03
       FReactionHist 14 : 3 OLD: 2.946149e+06 NEW: 2.947254e+06
       FReactionHist 14 : 4 OLD: -5.653142e+04 NEW: -1.075868e+05
       FReactionHist 14 : 5 OLD: 9.839289e+05 NEW: 1.828522e+05
       FReactionHist 14 : 6 OLD: -5.029699e+04 NEW: -6.158693e+04
       FReactionHist 15 : 1 OLD: 6.152321e+04 NEW: 1.459671e+04
       FReactionHist 15 : 2 OLD: 4.844859e+03 NEW: 9.973116e+03
       FReactionHist 15 : 3 OLD: 3.057770e+06 NEW: 3.092146e+06
       FReactionHist 15 : 4 OLD: -6.372762e+04 NEW: -1.263447e+05
       FReactionHist 15 : 5 OLD: 1.143327e+06 NEW: 2.459746e+05
       FReactionHist 15 : 6 OLD: -6.014461e+04 NEW: -7.580648e+04
       FReactionHist 16 : 1 OLD: 6.731060e+04 NEW: 2.310167e+04
       FReactionHist 16 : 2 OLD: 8.376310e+03 NEW: 1.359573e+04
       FReactionHist 16 : 3 OLD: 3.153126e+06 NEW: 3.217362e+06
       FReactionHist 16 : 4 OLD: -7.872475e+04 NEW: -1.455402e+05
       FReactionHist 16 : 5 OLD: 1.305007e+06 NEW: 3.327594e+05
       FReactionHist 16 : 6 OLD: -7.497424e+04 NEW: -8.980899e+04
       FReactionHist 17 : 1 OLD: 8.230119e+04 NEW: 3.559492e+04
       FReactionHist 17 : 2 OLD: 1.024532e+04 NEW: 1.613094e+04
       FReactionHist 17 : 3 OLD: 3.230910e+06 NEW: 3.317189e+06
       FReactionHist 17 : 4 OLD: -8.822670e+04 NEW: -1.589467e+05
       FReactionHist 17 : 5 OLD: 1.510418e+06 NEW: 4.403681e+05
       FReactionHist 17 : 6 OLD: -9.765531e+04 NEW: -1.045568e+05
       FReactionHist 18 : 1 OLD: 1.065081e+05 NEW: 4.890724e+04
       FReactionHist 18 : 2 OLD: 7.938057e+03 NEW: 1.515036e+04
       FReactionHist 18 : 3 OLD: 3.273443e+06 NEW: 3.383314e+06
       FReactionHist 18 : 4 OLD: -8.456955e+04 NEW: -1.591818e+05
       FReactionHist 18 : 5 OLD: 1.764005e+06 NEW: 5.515840e+05
       FReactionHist 18 : 6 OLD: -1.227681e+05 NEW: -1.220729e+05
       FReactionHist 19 : 1 OLD: 1.394250e+05 NEW: 5.892548e+04
       FReactionHist 19 : 2 OLD: 4.080830e+03 NEW: 1.194680e+04
       FReactionHist 19 : 3 OLD: 3.285289e+06 NEW: 3.431389e+06
       FReactionHist 19 : 4 OLD: -7.720880e+04 NEW: -1.468923e+05
       FReactionHist 19 : 5 OLD: 2.048196e+06 NEW: 6.505188e+05
       FReactionHist 19 : 6 OLD: -1.418991e+05 NEW: -1.430683e+05
       FReactionHist 20 : 1 OLD: 1.754878e+05 NEW: 6.606156e+04
       FReactionHist 20 : 2 OLD: 4.148639e+03 NEW: 9.146954e+03
       FReactionHist 20 : 3 OLD: 3.268342e+06 NEW: 3.460529e+06
       FReactionHist 20 : 4 OLD: -8.418677e+04 NEW: -1.353644e+05
       FReactionHist 20 : 5 OLD: 2.308836e+06 NEW: 7.252780e+05
       FReactionHist 20 : 6 OLD: -1.498487e+05 NEW: -1.638891e+05
       FReactionHist 21 : 1 OLD: 1.920116e+05 NEW: 6.980445e+04
       FReactionHist 21 : 2 OLD: 9.263111e+03 NEW: 7.586620e+03
       FReactionHist 21 : 3 OLD: 3.214659e+06 NEW: 3.446954e+06
       FReactionHist 21 : 4 OLD: -1.081526e+05 NEW: -1.279956e+05
       FReactionHist 21 : 5 OLD: 2.448197e+06 NEW: 7.706747e+05
       FReactionHist 21 : 6 OLD: -1.464631e+05 NEW: -1.820063e+05
       FReactionHist 22 : 1 OLD: 1.718877e+05 NEW: 6.959357e+04
       FReactionHist 22 : 2 OLD: 1.438798e+04 NEW: 7.055472e+03
       FReactionHist 22 : 3 OLD: 3.118671e+06 NEW: 3.378077e+06
       FReactionHist 22 : 4 OLD: -1.320333e+05 NEW: -1.256738e+05
       FReactionHist 22 : 5 OLD: 2.423031e+06 NEW: 7.816212e+05
       FReactionHist 22 : 6 OLD: -1.365775e+05 NEW: -1.948206e+05
       FReactionHist 23 : 1 OLD: 1.338156e+05 NEW: 6.428307e+04
       FReactionHist 23 : 2 OLD: 1.428200e+04 NEW: 6.611773e+03
       FReactionHist 23 : 3 OLD: 2.979634e+06 NEW: 3.262467e+06
       FReactionHist 23 : 4 OLD: -1.393290e+05 NEW: -1.296719e+05
       FReactionHist 23 : 5 OLD: 2.306570e+06 NEW: 7.536694e+05
       FReactionHist 23 : 6 OLD: -1.308751e+05 NEW: -2.027563e+05
       FReactionHist 24 : 1 OLD: 1.124152e+05 NEW: 5.446290e+04
       FReactionHist 24 : 2 OLD: 1.084510e+04 NEW: 7.838245e+03
       FReactionHist 24 : 3 OLD: 2.817431e+06 NEW: 3.128618e+06
       FReactionHist 24 : 4 OLD: -1.336725e+05 NEW: -1.379551e+05
       FReactionHist 24 : 5 OLD: 2.216584e+06 NEW: 6.927067e+05
       FReactionHist 24 : 6 OLD: -1.385456e+05 NEW: -2.081544e+05
       FReactionHist 25 : 1 OLD: 1.074521e+05 NEW: 4.191637e+04
       FReactionHist 25 : 2 OLD: 9.615636e+03 NEW: 1.012904e+04
       FReactionHist 25 : 3 OLD: 2.657899e+06 NEW: 3.001137e+06
       FReactionHist 25 : 4 OLD: -1.309875e+05 NEW: -1.476498e+05
       FReactionHist 25 : 5 OLD: 2.190722e+06 NEW: 6.092062e+05
       FReactionHist 25 : 6 OLD: -1.564494e+05 NEW: -2.095117e+05
       FReactionHist 26 : 1 OLD: 9.848444e+04 NEW: 2.854227e+04
       FReactionHist 26 : 2 OLD: 1.120493e+04 NEW: 1.017854e+04
       FReactionHist 26 : 3 OLD: 2.527702e+06 NEW: 2.893734e+06
       FReactionHist 26 : 4 OLD: -1.361058e+05 NEW: -1.463325e+05
       FReactionHist 26 : 5 OLD: 2.196934e+06 NEW: 5.174493e+05
       FReactionHist 26 : 6 OLD: -1.715531e+05 NEW: -2.053447e+05
       FReactionHist 27 : 1 OLD: 9.141744e+04 NEW: 1.655450e+04
       FReactionHist 27 : 2 OLD: 1.228409e+04 NEW: 5.496342e+03
       FReactionHist 27 : 3 OLD: 2.462574e+06 NEW: 2.827094e+06
       FReactionHist 27 : 4 OLD: -1.417384e+05 NEW: -1.275171e+05
       FReactionHist 27 : 5 OLD: 2.244657e+06 NEW: 4.335818e+05
       FReactionHist 27 : 6 OLD: -1.768987e+05 NEW: -2.027181e+05
       FReactionHist 28 : 1 OLD: 1.038709e+05 NEW: 7.575257e+03
       FReactionHist 28 : 2 OLD: 1.210098e+04 NEW: -1.214369e+03
       FReactionHist 28 : 3 OLD: 2.444617e+06 NEW: 2.785836e+06
       FReactionHist 28 : 4 OLD: -1.425837e+05 NEW: -1.046065e+05
       FReactionHist 28 : 5 OLD: 2.359731e+06 NEW: 3.693229e+05
       FReactionHist 28 : 6 OLD: -1.750634e+05 NEW: -2.082729e+05
       FReactionHist 29 : 1 OLD: 1.249837e+05 NEW: 3.117646e+03
       FReactionHist 29 : 2 OLD: 1.114729e+04 NEW: -4.941707e+03
       FReactionHist 29 : 3 OLD: 2.393481e+06 NEW: 2.708152e+06
       FReactionHist 29 : 4 OLD: -1.380475e+05 NEW: -9.567900e+04
       FReactionHist 29 : 5 OLD: 2.501623e+06 NEW: 3.313772e+05
       FReactionHist 29 : 6 OLD: -1.721517e+05 NEW: -2.264390e+05
       FReactionHist 30 : 1 OLD: 1.322929e+05 NEW: 1.966503e+03
       FReactionHist 30 : 2 OLD: 9.029000e+03 NEW: -5.165464e+03
       FReactionHist 30 : 3 OLD: 2.263661e+06 NEW: 2.555028e+06
       FReactionHist 30 : 4 OLD: -1.313334e+05 NEW: -1.138951e+05
       FReactionHist 30 : 5 OLD: 2.599713e+06 NEW: 3.148437e+05
       FReactionHist 30 : 6 OLD: -1.753254e+05 NEW: -2.572756e+05
       FReactionHist 31 : 1 OLD: 1.269810e+05 NEW: 8.560605e+02
       FReactionHist 31 : 2 OLD: 8.251867e+03 NEW: 1.569149e+02
       FReactionHist 31 : 3 OLD: 2.069228e+06 NEW: 2.330319e+06
       FReactionHist 31 : 4 OLD: -1.302810e+05 NEW: -1.674232e+05
       FReactionHist 31 : 5 OLD: 2.630570e+06 NEW: 3.059350e+05
       FReactionHist 31 : 6 OLD: -1.923637e+05 NEW: -2.972752e+05
       FReactionHist 32 : 1 OLD: 1.167076e+05 NEW: -2.105256e+03
       FReactionHist 32 : 2 OLD: 1.051775e+04 NEW: 1.107322e+04
       FReactionHist 32 : 3 OLD: 1.866105e+06 NEW: 2.085434e+06
       FReactionHist 32 : 4 OLD: -1.376543e+05 NEW: -2.511601e+05
       FReactionHist 32 : 5 OLD: 2.619023e+06 NEW: 2.997768e+05
       FReactionHist 32 : 6 OLD: -2.250440e+05 NEW: -3.438521e+05
       FReactionHist 33 : 1 OLD: 1.055757e+05 NEW: -3.305944e+03
       FReactionHist 33 : 2 OLD: 1.276755e+04 NEW: 2.434460e+04
       FReactionHist 33 : 3 OLD: 1.707179e+06 NEW: 1.875745e+06
       FReactionHist 33 : 4 OLD: -1.436184e+05 NEW: -3.493647e+05
       FReactionHist 33 : 5 OLD: 2.593474e+06 NEW: 3.001739e+05
       FReactionHist 33 : 6 OLD: -2.657375e+05 NEW: -3.954048e+05
       FReactionHist 34 : 1 OLD: 9.385162e+04 NEW: -3.017913e+03
       FReactionHist 34 : 2 OLD: 1.134134e+04 NEW: 3.615637e+04
       FReactionHist 34 : 3 OLD: 1.575011e+06 NEW: 1.689663e+06
       FReactionHist 34 : 4 OLD: -1.368330e+05 NEW: -4.406069e+05
       FReactionHist 34 : 5 OLD: 2.576019e+06 NEW: 3.079533e+05
       FReactionHist 34 : 6 OLD: -3.041237e+05 NEW: -4.498193e+05
       FReactionHist 35 : 1 OLD: 8.637810e+04 NEW: -3.915780e+03
       FReactionHist 35 : 2 OLD: 6.853035e+03 NEW: 4.494647e+04
       FReactionHist 35 : 3 OLD: 1.411005e+06 NEW: 1.485053e+06
       FReactionHist 35 : 4 OLD: -1.185265e+05 NEW: -5.047929e+05
       FReactionHist 35 : 5 OLD: 2.604190e+06 NEW: 3.207633e+05
       FReactionHist 35 : 6 OLD: -3.320457e+05 NEW: -4.999959e+05
       FReactionHist 36 : 1 OLD: 9.490298e+04 NEW: -3.999809e+03
       FReactionHist 36 : 2 OLD: 2.280761e+03 NEW: 4.732014e+04
       FReactionHist 36 : 3 OLD: 1.238541e+06 NEW: 1.283351e+06
       FReactionHist 36 : 4 OLD: -1.021918e+05 NEW: -5.254068e+05
       FReactionHist 36 : 5 OLD: 2.724584e+06 NEW: 3.453704e+05
       FReactionHist 36 : 6 OLD: -3.432683e+05 NEW: -5.348039e+05
       FReactionHist 37 : 1 OLD: 1.268966e+05 NEW: 4.327883e+02
       FReactionHist 37 : 2 OLD: 1.078024e+03 NEW: 4.145716e+04
       FReactionHist 37 : 3 OLD: 1.135310e+06 NEW: 1.141798e+06
       FReactionHist 37 : 4 OLD: -1.033049e+05 NEW: -4.973126e+05
       FReactionHist 37 : 5 OLD: 2.944936e+06 NEW: 3.889680e+05
       FReactionHist 37 : 6 OLD: -3.360152e+05 NEW: -5.497202e+05
       FReactionHist 38 : 1 OLD: 1.668647e+05 NEW: 6.455215e+03
       FReactionHist 38 : 2 OLD: 4.840742e+03 NEW: 3.073606e+04
       FReactionHist 38 : 3 OLD: 1.110722e+06 NEW: 1.063493e+06
       FReactionHist 38 : 4 OLD: -1.257647e+05 NEW: -4.308142e+05
       FReactionHist 38 : 5 OLD: 3.191046e+06 NEW: 4.465270e+05
       FReactionHist 38 : 6 OLD: -3.134078e+05 NEW: -5.478449e+05
       FReactionHist 39 : 1 OLD: 1.856342e+05 NEW: 1.120922e+04
       FReactionHist 39 : 2 OLD: 1.001564e+04 NEW: 1.860249e+04
       FReactionHist 39 : 3 OLD: 1.133153e+06 NEW: 1.015410e+06
       FReactionHist 39 : 4 OLD: -1.569109e+05 NEW: -3.464264e+05
       FReactionHist 39 : 5 OLD: 3.362434e+06 NEW: 5.109910e+05
       FReactionHist 39 : 6 OLD: -2.778774e+05 NEW: -5.303103e+05
       FReactionHist 40 : 1 OLD: 1.800385e+05 NEW: 1.770121e+04
       FReactionHist 40 : 2 OLD: 1.141842e+04 NEW: 6.958042e+03
       FReactionHist 40 : 3 OLD: 1.211304e+06 NEW: 1.012291e+06
       FReactionHist 40 : 4 OLD: -1.805170e+05 NEW: -2.605901e+05
       FReactionHist 40 : 5 OLD: 3.441220e+06 NEW: 5.838284e+05
       FReactionHist 40 : 6 OLD: -2.376497e+05 NEW: -5.047827e+05
       FReactionHist 41 : 1 OLD: 1.760826e+05 NEW: 2.543339e+04
       FReactionHist 41 : 2 OLD: 9.209623e+03 NEW: -2.965024e+03
       FReactionHist 41 : 3 OLD: 1.361596e+06 NEW: 1.094520e+06
       FReactionHist 41 : 4 OLD: -1.971051e+05 NEW: -1.868081e+05
       FReactionHist 41 : 5 OLD: 3.502013e+06 NEW: 6.652711e+05
       FReactionHist 41 : 6 OLD: -2.110962e+05 NEW: -4.835328e+05
       FReactionHist 42 : 1 OLD: 1.915293e+05 NEW: 3.287187e+04
       FReactionHist 42 : 2 OLD: 9.189790e+03 NEW: -8.907736e+03
       FReactionHist 42 : 3 OLD: 1.564778e+06 NEW: 1.250771e+06
       FReactionHist 42 : 4 OLD: -2.244041e+05 NEW: -1.394062e+05
       FReactionHist 42 : 5 OLD: 3.603496e+06 NEW: 7.522514e+05
       FReactionHist 42 : 6 OLD: -2.116156e+05 NEW: -4.748309e+05
       FReactionHist 43 : 1 OLD: 2.134499e+05 NEW: 4.134061e+04
       FReactionHist 43 : 2 OLD: 1.493455e+04 NEW: -1.019436e+04
       FReactionHist 43 : 3 OLD: 1.747241e+06 NEW: 1.406737e+06
       FReactionHist 43 : 4 OLD: -2.725513e+05 NEW: -1.239222e+05
       FReactionHist 43 : 5 OLD: 3.718355e+06 NEW: 8.447095e+05
       FReactionHist 43 : 6 OLD: -2.328435e+05 NEW: -4.798564e+05
       FReactionHist 44 : 1 OLD: 2.279972e+05 NEW: 5.183048e+04
       FReactionHist 44 : 2 OLD: 2.318336e+04 NEW: -7.247365e+03
       FReactionHist 44 : 3 OLD: 1.852176e+06 NEW: 1.512318e+06
       FReactionHist 44 : 4 OLD: -3.291470e+05 NEW: -1.399751e+05
       FReactionHist 44 : 5 OLD: 3.794033e+06 NEW: 9.408992e+05
       FReactionHist 44 : 6 OLD: -2.581807e+05 NEW: -4.966218e+05
       FReactionHist 45 : 1 OLD: 2.346854e+05 NEW: 6.056429e+04
       FReactionHist 45 : 2 OLD: 2.748708e+04 NEW: -1.662683e+03
       FReactionHist 45 : 3 OLD: 1.926817e+06 NEW: 1.605368e+06
       FReactionHist 45 : 4 OLD: -3.737321e+05 NEW: -1.829237e+05
       FReactionHist 45 : 5 OLD: 3.795662e+06 NEW: 1.029989e+06
       FReactionHist 45 : 6 OLD: -2.786692e+05 NEW: -5.244541e+05
       FReactionHist 46 : 1 OLD: 2.270696e+05 NEW: 6.737566e+04
       FReactionHist 46 : 2 OLD: 2.728351e+04 NEW: 6.551262e+03
       FReactionHist 46 : 3 OLD: 2.027003e+06 NEW: 1.733924e+06
       FReactionHist 46 : 4 OLD: -4.032465e+05 NEW: -2.426420e+05
       FReactionHist 46 : 5 OLD: 3.693849e+06 NEW: 1.105250e+06
       FReactionHist 46 : 6 OLD: -3.032877e+05 NEW: -5.694537e+05
       FReactionHist 47 : 1 OLD: 1.968345e+05 NEW: 7.289361e+04
       FReactionHist 47 : 2 OLD: 2.843782e+04 NEW: 1.430780e+04
       FReactionHist 47 : 3 OLD: 2.136777e+06 NEW: 1.884032e+06
       FReactionHist 47 : 4 OLD: -4.331840e+05 NEW: -3.056293e+05
       FReactionHist 47 : 5 OLD: 3.495431e+06 NEW: 1.160607e+06
       FReactionHist 47 : 6 OLD: -3.480500e+05 NEW: -6.369610e+05
       FReactionHist 48 : 1 OLD: 1.658532e+05 NEW: 7.384912e+04
       FReactionHist 48 : 2 OLD: 3.391308e+04 NEW: 1.839606e+04
       FReactionHist 48 : 3 OLD: 2.234898e+06 NEW: 2.031959e+06
       FReactionHist 48 : 4 OLD: -4.698479e+05 NEW: -3.571466e+05
       FReactionHist 48 : 5 OLD: 3.270410e+06 NEW: 1.186278e+06
       FReactionHist 48 : 6 OLD: -4.089249e+05 NEW: -7.218311e+05
       FReactionHist 49 : 1 OLD: 1.534348e+05 NEW: 7.006952e+04
       FReactionHist 49 : 2 OLD: 3.852997e+04 NEW: 1.868355e+04
       FReactionHist 49 : 3 OLD: 2.328983e+06 NEW: 2.193721e+06
       FReactionHist 49 : 4 OLD: -4.978654e+05 NEW: -3.936374e+05
       FReactionHist 49 : 5 OLD: 3.081498e+06 NEW: 1.182568e+06
       FReactionHist 49 : 6 OLD: -4.587724e+05 NEW: -8.090820e+05
       FReactionHist 50 : 1 OLD: 1.448550e+05 NEW: 6.525813e+04
       FReactionHist 50 : 2 OLD: 3.742459e+04 NEW: 1.790686e+04
       FReactionHist 50 : 3 OLD: 2.429362e+06 NEW: 2.382011e+06
       FReactionHist 50 : 4 OLD: -5.026903e+05 NEW: -4.259125e+05
       FReactionHist 50 : 5 OLD: 2.915128e+06 NEW: 1.154862e+06
       FReactionHist 50 : 6 OLD: -4.778675e+05 NEW: -8.803484e+05
       FReactionHist 51 : 1 OLD: 1.276172e+05 NEW: 5.785529e+04
       FReactionHist 51 : 2 OLD: 3.283271e+04 NEW: 1.915120e+04
       FReactionHist 51 : 3 OLD: 2.543811e+06 NEW: 2.588247e+06
       FReactionHist 51 : 4 OLD: -4.905020e+05 NEW: -4.636258e+05
       FReactionHist 51 : 5 OLD: 2.737870e+06 NEW: 1.105042e+06
       FReactionHist 51 : 6 OLD: -4.717114e+05 NEW: -9.226510e+05
       
This is what it looks like with the two way coupling turned off
      FReactionHist 2 : 1 OLD: -7.142141e+02 NEW: -3.293558e+02
       FReactionHist 2 : 2 OLD: -1.076061e+02 NEW: -2.985060e+02
       FReactionHist 2 : 3 OLD: 1.515958e+05 NEW: 1.506536e+05
       FReactionHist 2 : 4 OLD: 7.069546e+01 NEW: -7.408226e+02
       FReactionHist 2 : 5 OLD: -7.420892e+02 NEW: -6.213884e+01
       FReactionHist 2 : 6 OLD: 1.359528e+02 NEW: 1.863842e+02
       FReactionHist 3 : 1 OLD: -3.359312e+03 NEW: -1.021057e+03
       FReactionHist 3 : 2 OLD: -4.588144e+02 NEW: -1.670599e+02
       FReactionHist 3 : 3 OLD: 4.326783e+05 NEW: 4.264231e+05
       FReactionHist 3 : 4 OLD: -1.020519e+03 NEW: -8.938611e+03
       FReactionHist 3 : 5 OLD: 3.280347e+03 NEW: 4.734440e+03
       FReactionHist 3 : 6 OLD: 1.258519e+03 NEW: 1.440251e+03
       FReactionHist 4 : 1 OLD: -3.979357e+03 NEW: 1.438165e+03
       FReactionHist 4 : 2 OLD: -1.340640e+02 NEW: 3.528589e+03
       FReactionHist 4 : 3 OLD: 7.319887e+05 NEW: 7.162688e+05
       FReactionHist 4 : 4 OLD: -6.307394e+03 NEW: -2.884216e+04
       FReactionHist 4 : 5 OLD: 2.314380e+04 NEW: 2.058358e+04
       FReactionHist 4 : 6 OLD: 3.744007e+03 NEW: 2.469860e+03
       FReactionHist 5 : 1 OLD: -1.750950e+03 NEW: 4.991264e+03
       FReactionHist 5 : 2 OLD: 1.599643e+03 NEW: 7.504147e+03
       FReactionHist 5 : 3 OLD: 9.992354e+05 NEW: 9.775499e+05
       FReactionHist 5 : 4 OLD: -1.744892e+04 NEW: -5.062069e+04
       FReactionHist 5 : 5 OLD: 6.147387e+04 NEW: 3.932369e+04
       FReactionHist 5 : 6 OLD: 3.374528e+03 NEW: -3.505159e+03
       FReactionHist 6 : 1 OLD: 2.905970e+03 NEW: 5.156095e+03
       FReactionHist 6 : 2 OLD: 4.352465e+03 NEW: 8.240090e+03
       FReactionHist 6 : 3 OLD: 1.273110e+06 NEW: 1.248294e+06
       FReactionHist 6 : 4 OLD: -3.212057e+04 NEW: -6.551485e+04
       FReactionHist 6 : 5 OLD: 1.179283e+05 NEW: 5.385450e+04
       FReactionHist 6 : 6 OLD: -2.245956e+03 NEW: -1.167963e+04
       FReactionHist 7 : 1 OLD: 9.451359e+03 NEW: 5.401021e+03
       FReactionHist 7 : 2 OLD: 6.298100e+03 NEW: 9.057147e+03
       FReactionHist 7 : 3 OLD: 1.535036e+06 NEW: 1.505661e+06
       FReactionHist 7 : 4 OLD: -4.252528e+04 NEW: -7.577026e+04
       FReactionHist 7 : 5 OLD: 1.766629e+05 NEW: 6.439362e+04
       FReactionHist 7 : 6 OLD: -9.761377e+03 NEW: -1.419995e+04
       FReactionHist 8 : 1 OLD: 7.140729e+03 NEW: 5.981078e+03
       FReactionHist 8 : 2 OLD: 5.190312e+03 NEW: 9.405317e+03
       FReactionHist 8 : 3 OLD: 1.800449e+06 NEW: 1.762505e+06
       FReactionHist 8 : 4 OLD: -4.293471e+04 NEW: -7.968008e+04
       FReactionHist 8 : 5 OLD: 2.083231e+05 NEW: 7.335987e+04
       FReactionHist 8 : 6 OLD: -1.689814e+04 NEW: -1.682180e+04
       FReactionHist 9 : 1 OLD: -5.777089e+03 NEW: 5.320609e+03
       FReactionHist 9 : 2 OLD: 3.081757e+03 NEW: 7.380429e+03
       FReactionHist 9 : 3 OLD: 2.059662e+06 NEW: 2.007919e+06
       FReactionHist 9 : 4 OLD: -4.110436e+04 NEW: -7.763024e+04
       FReactionHist 9 : 5 OLD: 2.195080e+05 NEW: 8.081772e+04
       FReactionHist 9 : 6 OLD: -2.325660e+04 NEW: -2.133270e+04
       FReactionHist 10 : 1 OLD: -1.532398e+04 NEW: 4.446682e+03
       FReactionHist 10 : 2 OLD: 4.241030e+03 NEW: 5.412073e+03
       FReactionHist 10 : 3 OLD: 2.292043e+06 NEW: 2.230615e+06
       FReactionHist 10 : 4 OLD: -4.820738e+04 NEW: -7.608259e+04
       FReactionHist 10 : 5 OLD: 2.564952e+05 NEW: 8.467557e+04
       FReactionHist 10 : 6 OLD: -2.816460e+04 NEW: -2.597245e+04
       FReactionHist 11 : 1 OLD: -8.625607e+03 NEW: 3.714157e+03
       FReactionHist 11 : 2 OLD: 7.871172e+03 NEW: 5.908188e+03
       FReactionHist 11 : 3 OLD: 2.496174e+06 NEW: 2.435181e+06
       FReactionHist 11 : 4 OLD: -6.118161e+04 NEW: -7.832971e+04
       FReactionHist 11 : 5 OLD: 3.693550e+05 NEW: 9.202166e+04
       FReactionHist 11 : 6 OLD: -3.092693e+04 NEW: -3.114141e+04
       FReactionHist 12 : 1 OLD: 1.378745e+04 NEW: 3.535617e+03
       FReactionHist 12 : 2 OLD: 8.676335e+03 NEW: 5.862247e+03
       FReactionHist 12 : 3 OLD: 2.661933e+06 NEW: 2.611734e+06
       FReactionHist 12 : 4 OLD: -6.678975e+04 NEW: -8.553416e+04
       FReactionHist 12 : 5 OLD: 5.642306e+05 NEW: 1.073617e+05
       FReactionHist 12 : 6 OLD: -3.419050e+04 NEW: -3.783606e+04
       FReactionHist 13 : 1 OLD: 4.188317e+04 NEW: 5.971462e+03
       FReactionHist 13 : 2 OLD: 5.949264e+03 NEW: 6.339923e+03
       FReactionHist 13 : 3 OLD: 2.809592e+06 NEW: 2.781024e+06
       FReactionHist 13 : 4 OLD: -6.122785e+04 NEW: -9.479565e+04
       FReactionHist 13 : 5 OLD: 7.879436e+05 NEW: 1.366412e+05
       FReactionHist 13 : 6 OLD: -4.106554e+04 NEW: -4.785621e+04
       FReactionHist 14 : 1 OLD: 5.831440e+04 NEW: 9.746277e+03
       FReactionHist 14 : 2 OLD: 3.728332e+03 NEW: 8.077783e+03
       FReactionHist 14 : 3 OLD: 2.946149e+06 NEW: 2.947301e+06
       FReactionHist 14 : 4 OLD: -5.653142e+04 NEW: -1.078427e+05
       FReactionHist 14 : 5 OLD: 9.839289e+05 NEW: 1.829966e+05
       FReactionHist 14 : 6 OLD: -5.029699e+04 NEW: -6.180139e+04
       FReactionHist 15 : 1 OLD: 6.152321e+04 NEW: 1.460003e+04
       FReactionHist 15 : 2 OLD: 4.844859e+03 NEW: 1.002343e+04
       FReactionHist 15 : 3 OLD: 3.057770e+06 NEW: 3.092214e+06
       FReactionHist 15 : 4 OLD: -6.372762e+04 NEW: -1.267618e+05
       FReactionHist 15 : 5 OLD: 1.143327e+06 NEW: 2.462694e+05
       FReactionHist 15 : 6 OLD: -6.014461e+04 NEW: -7.611258e+04
       FReactionHist 16 : 1 OLD: 6.731060e+04 NEW: 2.310879e+04
       FReactionHist 16 : 2 OLD: 8.376310e+03 NEW: 1.365647e+04
       FReactionHist 16 : 3 OLD: 3.153126e+06 NEW: 3.217471e+06
       FReactionHist 16 : 4 OLD: -7.872475e+04 NEW: -1.460906e+05
       FReactionHist 16 : 5 OLD: 1.305007e+06 NEW: 3.332276e+05
       FReactionHist 16 : 6 OLD: -7.497424e+04 NEW: -9.022247e+04
       FReactionHist 17 : 1 OLD: 8.230119e+04 NEW: 3.560819e+04
       FReactionHist 17 : 2 OLD: 1.024532e+04 NEW: 1.619458e+04
       FReactionHist 17 : 3 OLD: 3.230910e+06 NEW: 3.317351e+06
       FReactionHist 17 : 4 OLD: -8.822670e+04 NEW: -1.596060e+05
       FReactionHist 17 : 5 OLD: 1.510418e+06 NEW: 4.410282e+05
       FReactionHist 17 : 6 OLD: -9.765531e+04 NEW: -1.051021e+05
       FReactionHist 18 : 1 OLD: 1.065081e+05 NEW: 4.892000e+04
       FReactionHist 18 : 2 OLD: 7.938057e+03 NEW: 1.521805e+04
       FReactionHist 18 : 3 OLD: 3.273443e+06 NEW: 3.383536e+06
       FReactionHist 18 : 4 OLD: -8.456955e+04 NEW: -1.599492e+05
       FReactionHist 18 : 5 OLD: 1.764005e+06 NEW: 5.524574e+05
       FReactionHist 18 : 6 OLD: -1.227681e+05 NEW: -1.227837e+05
       FReactionHist 19 : 1 OLD: 1.394250e+05 NEW: 5.893436e+04
       FReactionHist 19 : 2 OLD: 4.080830e+03 NEW: 1.201771e+04
       FReactionHist 19 : 3 OLD: 3.285289e+06 NEW: 3.431674e+06
       FReactionHist 19 : 4 OLD: -7.720880e+04 NEW: -1.477771e+05
       FReactionHist 19 : 5 OLD: 2.048196e+06 NEW: 6.516360e+05
       FReactionHist 19 : 6 OLD: -1.418991e+05 NEW: -1.439673e+05
       FReactionHist 20 : 1 OLD: 1.754878e+05 NEW: 6.607988e+04
       FReactionHist 20 : 2 OLD: 4.148639e+03 NEW: 9.221371e+03
       FReactionHist 20 : 3 OLD: 3.268342e+06 NEW: 3.460878e+06
       FReactionHist 20 : 4 OLD: -8.418677e+04 NEW: -1.363832e+05
       FReactionHist 20 : 5 OLD: 2.308836e+06 NEW: 7.267251e+05
       FReactionHist 20 : 6 OLD: -1.498487e+05 NEW: -1.649952e+05
       FReactionHist 21 : 1 OLD: 1.920116e+05 NEW: 6.984112e+04
       FReactionHist 21 : 2 OLD: 9.263111e+03 NEW: 7.668588e+03
       FReactionHist 21 : 3 OLD: 3.214659e+06 NEW: 3.447396e+06
       FReactionHist 21 : 4 OLD: -1.081526e+05 NEW: -1.291922e+05
       FReactionHist 21 : 5 OLD: 2.448197e+06 NEW: 7.725666e+05
       FReactionHist 21 : 6 OLD: -1.464631e+05 NEW: -1.833528e+05
       FReactionHist 22 : 1 OLD: 1.718877e+05 NEW: 6.966380e+04
       FReactionHist 22 : 2 OLD: 1.438798e+04 NEW: 7.158176e+03
       FReactionHist 22 : 3 OLD: 3.118671e+06 NEW: 3.378642e+06
       FReactionHist 22 : 4 OLD: -1.320333e+05 NEW: -1.270919e+05
       FReactionHist 22 : 5 OLD: 2.423031e+06 NEW: 7.840858e+05
       FReactionHist 22 : 6 OLD: -1.365775e+05 NEW: -1.964285e+05
       FReactionHist 23 : 1 OLD: 1.338156e+05 NEW: 6.439466e+04
       FReactionHist 23 : 2 OLD: 1.428200e+04 NEW: 6.730417e+03
       FReactionHist 23 : 3 OLD: 2.979634e+06 NEW: 3.263174e+06
       FReactionHist 23 : 4 OLD: -1.393290e+05 NEW: -1.313254e+05
       FReactionHist 23 : 5 OLD: 2.306570e+06 NEW: 7.568448e+05
       FReactionHist 23 : 6 OLD: -1.308751e+05 NEW: -2.046734e+05
       FReactionHist 24 : 1 OLD: 1.124152e+05 NEW: 5.461685e+04
       FReactionHist 24 : 2 OLD: 1.084510e+04 NEW: 7.967329e+03
       FReactionHist 24 : 3 OLD: 2.817431e+06 NEW: 3.129533e+06
       FReactionHist 24 : 4 OLD: -1.336725e+05 NEW: -1.398803e+05
       FReactionHist 24 : 5 OLD: 2.216584e+06 NEW: 6.967837e+05
       FReactionHist 24 : 6 OLD: -1.385456e+05 NEW: -2.104513e+05
       FReactionHist 25 : 1 OLD: 1.074521e+05 NEW: 4.215797e+04
       FReactionHist 25 : 2 OLD: 9.615636e+03 NEW: 1.029572e+04
       FReactionHist 25 : 3 OLD: 2.657899e+06 NEW: 3.002316e+06
       FReactionHist 25 : 4 OLD: -1.309875e+05 NEW: -1.499449e+05
       FReactionHist 25 : 5 OLD: 2.190722e+06 NEW: 6.144876e+05
       FReactionHist 25 : 6 OLD: -1.564494e+05 NEW: -2.122615e+05
       FReactionHist 26 : 1 OLD: 9.848444e+04 NEW: 2.891777e+04
       FReactionHist 26 : 2 OLD: 1.120493e+04 NEW: 1.039502e+04
       FReactionHist 26 : 3 OLD: 2.527702e+06 NEW: 2.895189e+06
       FReactionHist 26 : 4 OLD: -1.361058e+05 NEW: -1.490763e+05
       FReactionHist 26 : 5 OLD: 2.196934e+06 NEW: 5.242600e+05
       FReactionHist 26 : 6 OLD: -1.715531e+05 NEW: -2.086727e+05
       FReactionHist 27 : 1 OLD: 9.141744e+04 NEW: 1.708117e+04
       FReactionHist 27 : 2 OLD: 1.228409e+04 NEW: 5.760522e+03
       FReactionHist 27 : 3 OLD: 2.462574e+06 NEW: 2.828900e+06
       FReactionHist 27 : 4 OLD: -1.417384e+05 NEW: -1.307234e+05
       FReactionHist 27 : 5 OLD: 2.244657e+06 NEW: 4.421649e+05
       FReactionHist 27 : 6 OLD: -1.768987e+05 NEW: -2.067098e+05
       FReactionHist 28 : 1 OLD: 1.038709e+05 NEW: 8.267777e+03
       FReactionHist 28 : 2 OLD: 1.210098e+04 NEW: -9.128435e+02
       FReactionHist 28 : 3 OLD: 2.444617e+06 NEW: 2.788055e+06
       FReactionHist 28 : 4 OLD: -1.425837e+05 NEW: -1.082194e+05
       FReactionHist 28 : 5 OLD: 2.359731e+06 NEW: 3.797901e+05
       FReactionHist 28 : 6 OLD: -1.750634e+05 NEW: -2.129350e+05
       FReactionHist 29 : 1 OLD: 1.249837e+05 NEW: 3.960788e+03
       FReactionHist 29 : 2 OLD: 1.114729e+04 NEW: -4.631775e+03
       FReactionHist 29 : 3 OLD: 2.393481e+06 NEW: 2.710813e+06
       FReactionHist 29 : 4 OLD: -1.380475e+05 NEW: -9.957077e+04
       FReactionHist 29 : 5 OLD: 2.501623e+06 NEW: 3.436765e+05
       FReactionHist 29 : 6 OLD: -1.721517e+05 NEW: -2.318136e+05
       FReactionHist 30 : 1 OLD: 1.322929e+05 NEW: 2.906720e+03
       FReactionHist 30 : 2 OLD: 9.029000e+03 NEW: -4.880835e+03
       FReactionHist 30 : 3 OLD: 2.263661e+06 NEW: 2.558182e+06
       FReactionHist 30 : 4 OLD: -1.313334e+05 NEW: -1.179696e+05
       FReactionHist 30 : 5 OLD: 2.599713e+06 NEW: 3.287670e+05
       FReactionHist 30 : 6 OLD: -1.753254e+05 NEW: -2.634362e+05
       FReactionHist 31 : 1 OLD: 1.269810e+05 NEW: 1.853903e+03
       FReactionHist 31 : 2 OLD: 8.251867e+03 NEW: 4.198695e+02
       FReactionHist 31 : 3 OLD: 2.069228e+06 NEW: 2.333950e+06
       FReactionHist 31 : 4 OLD: -1.302810e+05 NEW: -1.717235e+05
       FReactionHist 31 : 5 OLD: 2.630570e+06 NEW: 3.212249e+05
       FReactionHist 31 : 6 OLD: -1.923637e+05 NEW: -3.043007e+05
       FReactionHist 32 : 1 OLD: 1.167076e+05 NEW: -1.079445e+03
       FReactionHist 32 : 2 OLD: 1.051775e+04 NEW: 1.135782e+04
       FReactionHist 32 : 3 OLD: 1.866105e+06 NEW: 2.089493e+06
       FReactionHist 32 : 4 OLD: -1.376543e+05 NEW: -2.558436e+05
       FReactionHist 32 : 5 OLD: 2.619023e+06 NEW: 3.162387e+05
       FReactionHist 32 : 6 OLD: -2.250440e+05 NEW: -3.518169e+05
       FReactionHist 33 : 1 OLD: 1.055757e+05 NEW: -2.272146e+03
       FReactionHist 33 : 2 OLD: 1.276755e+04 NEW: 2.467452e+04
       FReactionHist 33 : 3 OLD: 1.707179e+06 NEW: 1.880187e+06
       FReactionHist 33 : 4 OLD: -1.436184e+05 NEW: -3.545403e+05
       FReactionHist 33 : 5 OLD: 2.593474e+06 NEW: 3.176432e+05
       FReactionHist 33 : 6 OLD: -2.657375e+05 NEW: -4.042965e+05
       FReactionHist 34 : 1 OLD: 9.385162e+04 NEW: -1.997246e+03
       FReactionHist 34 : 2 OLD: 1.134134e+04 NEW: 3.650112e+04
       FReactionHist 34 : 3 OLD: 1.575011e+06 NEW: 1.694473e+06
       FReactionHist 34 : 4 OLD: -1.368330e+05 NEW: -4.462236e+05
       FReactionHist 34 : 5 OLD: 2.576019e+06 NEW: 3.263146e+05
       FReactionHist 34 : 6 OLD: -3.041237e+05 NEW: -4.596292e+05
       FReactionHist 35 : 1 OLD: 8.637810e+04 NEW: -2.935473e+03
       FReactionHist 35 : 2 OLD: 6.853035e+03 NEW: 4.527716e+04
       FReactionHist 35 : 3 OLD: 1.411005e+06 NEW: 1.490307e+06
       FReactionHist 35 : 4 OLD: -1.185265e+05 NEW: -5.108069e+05
       FReactionHist 35 : 5 OLD: 2.604190e+06 NEW: 3.399994e+05
       FReactionHist 35 : 6 OLD: -3.320457e+05 NEW: -5.107837e+05
       FReactionHist 36 : 1 OLD: 9.490298e+04 NEW: -3.049993e+03
       FReactionHist 36 : 2 OLD: 2.280761e+03 NEW: 4.766173e+04
       FReactionHist 36 : 3 OLD: 1.238541e+06 NEW: 1.289053e+06
       FReactionHist 36 : 4 OLD: -1.021918e+05 NEW: -5.319252e+05
       FReactionHist 36 : 5 OLD: 2.724584e+06 NEW: 3.655989e+05
       FReactionHist 36 : 6 OLD: -3.432683e+05 NEW: -5.466161e+05
       FReactionHist 37 : 1 OLD: 1.268966e+05 NEW: 1.391538e+03
       FReactionHist 37 : 2 OLD: 1.078024e+03 NEW: 4.183984e+04
       FReactionHist 37 : 3 OLD: 1.135310e+06 NEW: 1.147856e+06
       FReactionHist 37 : 4 OLD: -1.033049e+05 NEW: -5.044761e+05
       FReactionHist 37 : 5 OLD: 2.944936e+06 NEW: 4.105385e+05
       FReactionHist 37 : 6 OLD: -3.360152e+05 NEW: -5.626444e+05
       FReactionHist 38 : 1 OLD: 1.668647e+05 NEW: 7.488118e+03
       FReactionHist 38 : 2 OLD: 4.840742e+03 NEW: 3.115465e+04
       FReactionHist 38 : 3 OLD: 1.110722e+06 NEW: 1.069900e+06
       FReactionHist 38 : 4 OLD: -1.257647e+05 NEW: -4.387054e+05
       FReactionHist 38 : 5 OLD: 3.191046e+06 NEW: 4.698701e+05
       FReactionHist 38 : 6 OLD: -3.134078e+05 NEW: -5.620596e+05
       FReactionHist 39 : 1 OLD: 1.856342e+05 NEW: 1.236352e+04
       FReactionHist 39 : 2 OLD: 1.001564e+04 NEW: 1.903460e+04
       FReactionHist 39 : 3 OLD: 1.133153e+06 NEW: 1.022199e+06
       FReactionHist 39 : 4 OLD: -1.569109e+05 NEW: -3.551233e+05
       FReactionHist 39 : 5 OLD: 3.362434e+06 NEW: 5.364643e+05
       FReactionHist 39 : 6 OLD: -2.778774e+05 NEW: -5.460743e+05
       FReactionHist 40 : 1 OLD: 1.800385e+05 NEW: 1.898624e+04
       FReactionHist 40 : 2 OLD: 1.141842e+04 NEW: 7.429363e+03
       FReactionHist 40 : 3 OLD: 1.211304e+06 NEW: 1.019494e+06
       FReactionHist 40 : 4 OLD: -1.805170e+05 NEW: -2.703182e+05
       FReactionHist 40 : 5 OLD: 3.441220e+06 NEW: 6.116836e+05
       FReactionHist 40 : 6 OLD: -2.376497e+05 NEW: -5.224026e+05
       FReactionHist 41 : 1 OLD: 1.760826e+05 NEW: 2.685119e+04
       FReactionHist 41 : 2 OLD: 9.209623e+03 NEW: -2.383514e+03
       FReactionHist 41 : 3 OLD: 1.361596e+06 NEW: 1.102174e+06
       FReactionHist 41 : 4 OLD: -1.971051e+05 NEW: -1.979373e+05
       FReactionHist 41 : 5 OLD: 3.502013e+06 NEW: 6.955748e+05
       FReactionHist 41 : 6 OLD: -2.110962e+05 NEW: -5.031957e+05
       FReactionHist 42 : 1 OLD: 1.915293e+05 NEW: 3.440899e+04
       FReactionHist 42 : 2 OLD: 9.189790e+03 NEW: -8.168374e+03
       FReactionHist 42 : 3 OLD: 1.564778e+06 NEW: 1.258803e+06
       FReactionHist 42 : 4 OLD: -2.244041e+05 NEW: -1.521999e+05
       FReactionHist 42 : 5 OLD: 3.603496e+06 NEW: 7.849928e+05
       FReactionHist 42 : 6 OLD: -2.116156e+05 NEW: -4.965771e+05
       FReactionHist 43 : 1 OLD: 2.134499e+05 NEW: 4.296470e+04
       FReactionHist 43 : 2 OLD: 1.493455e+04 NEW: -9.313031e+03
       FReactionHist 43 : 3 OLD: 1.747241e+06 NEW: 1.415027e+06
       FReactionHist 43 : 4 OLD: -2.725513e+05 NEW: -1.384176e+05
       FReactionHist 43 : 5 OLD: 3.718355e+06 NEW: 8.798047e+05
       FReactionHist 43 : 6 OLD: -2.328435e+05 NEW: -5.037357e+05
       FReactionHist 44 : 1 OLD: 2.279972e+05 NEW: 5.352344e+04
       FReactionHist 44 : 2 OLD: 2.318336e+04 NEW: -6.247883e+03
       FReactionHist 44 : 3 OLD: 1.852176e+06 NEW: 1.520859e+06
       FReactionHist 44 : 4 OLD: -3.291470e+05 NEW: -1.561087e+05
       FReactionHist 44 : 5 OLD: 3.794033e+06 NEW: 9.782271e+05
       FReactionHist 44 : 6 OLD: -2.581807e+05 NEW: -5.226805e+05
       FReactionHist 45 : 1 OLD: 2.346854e+05 NEW: 6.231823e+04
       FReactionHist 45 : 2 OLD: 2.748708e+04 NEW: -5.355699e+02
       FReactionHist 45 : 3 OLD: 1.926817e+06 NEW: 1.614187e+06
       FReactionHist 45 : 4 OLD: -3.737321e+05 NEW: -2.005917e+05
       FReactionHist 45 : 5 OLD: 3.795662e+06 NEW: 1.069495e+06
       FReactionHist 45 : 6 OLD: -2.786692e+05 NEW: -5.526983e+05
       FReactionHist 46 : 1 OLD: 2.270696e+05 NEW: 6.916468e+04
       FReactionHist 46 : 2 OLD: 2.728351e+04 NEW: 7.786529e+03
       FReactionHist 46 : 3 OLD: 2.027003e+06 NEW: 1.743012e+06
       FReactionHist 46 : 4 OLD: -4.032465e+05 NEW: -2.615465e+05
       FReactionHist 46 : 5 OLD: 3.693849e+06 NEW: 1.146808e+06
       FReactionHist 46 : 6 OLD: -3.032877e+05 NEW: -5.998270e+05
       FReactionHist 47 : 1 OLD: 1.968345e+05 NEW: 7.467569e+04
       FReactionHist 47 : 2 OLD: 2.843782e+04 NEW: 1.554393e+04
       FReactionHist 47 : 3 OLD: 2.136777e+06 NEW: 1.893377e+06
       FReactionHist 47 : 4 OLD: -4.331840e+05 NEW: -3.252007e+05
       FReactionHist 47 : 5 OLD: 3.495431e+06 NEW: 1.204121e+06
       FReactionHist 47 : 6 OLD: -3.480500e+05 NEW: -6.693621e+05
       FReactionHist 48 : 1 OLD: 1.658532e+05 NEW: 7.562789e+04
       FReactionHist 48 : 2 OLD: 3.391308e+04 NEW: 1.950554e+04
       FReactionHist 48 : 3 OLD: 2.234898e+06 NEW: 2.041495e+06
       FReactionHist 48 : 4 OLD: -4.698479e+05 NEW: -3.768031e+05
       FReactionHist 48 : 5 OLD: 3.270410e+06 NEW: 1.231853e+06
       FReactionHist 48 : 6 OLD: -4.089249e+05 NEW: -7.562611e+05
       FReactionHist 49 : 1 OLD: 1.534348e+05 NEW: 7.188836e+04
       FReactionHist 49 : 2 OLD: 3.852997e+04 NEW: 1.963184e+04
       FReactionHist 49 : 3 OLD: 2.328983e+06 NEW: 2.203318e+06
       FReactionHist 49 : 4 OLD: -4.978654e+05 NEW: -4.131474e+05
       FReactionHist 49 : 5 OLD: 3.081498e+06 NEW: 1.230396e+06
       FReactionHist 49 : 6 OLD: -4.587724e+05 NEW: -8.456498e+05
       FReactionHist 50 : 1 OLD: 1.448550e+05 NEW: 6.712107e+04
       FReactionHist 50 : 2 OLD: 3.742459e+04 NEW: 1.873222e+04
       FReactionHist 50 : 3 OLD: 2.429362e+06 NEW: 2.391563e+06
       FReactionHist 50 : 4 OLD: -5.026903e+05 NEW: -4.455000e+05
       FReactionHist 50 : 5 OLD: 2.915128e+06 NEW: 1.205203e+06
       FReactionHist 50 : 6 OLD: -4.778675e+05 NEW: -9.191645e+05
       FReactionHist 51 : 1 OLD: 1.276172e+05 NEW: 5.979980e+04
       FReactionHist 51 : 2 OLD: 3.283271e+04 NEW: 1.994052e+04
       FReactionHist 51 : 3 OLD: 2.543811e+06 NEW: 2.597692e+06
       FReactionHist 51 : 4 OLD: -4.905020e+05 NEW: -4.838268e+05
       FReactionHist 51 : 5 OLD: 2.737870e+06 NEW: 1.158190e+06
       FReactionHist 51 : 6 OLD: -4.717114e+05 NEW: -9.638651e+05
       
Here is the difference between the New of the two sets:


-0.0004999999999881766
-0.0003999999999564352
0.0
-0.001099999999951251
-3.999999999848569e-5
0.00029999999998153726
-0.002000000000066393
-0.0011000000000080945
0.0
-0.015000000001236913
0.006999999999607098
0.0039999999999054126
-0.005000000000109139
0.0019999999999527063
0.0
-0.07999999999810825
0.040000000000873115
0.018000000000029104
-0.007999999999810825
0.012999999999919964
0.09999999997671694
-0.31000000000494765
0.20000000000436557
0.04899999999997817
-0.001999999999497959
0.06199999999989814
0.0
-0.9399999999950523
0.6200000000026193
0.09000000000014552
0.04399999999986903
0.19800000000032014
1.0
-2.3699999999953434
1.6200000000026193
0.06999999999970896
0.15100000000074942
0.47999999999956344
1.0
-5.139999999999418
3.5899999999965075
-0.23999999999796273
0.32999999999992724
0.975999999999658
2.0
-9.940000000002328
7.029999999998836
-1.2099999999991269
0.9059999999999491
1.9570000000003347
2.0
-15.849999999991269
11.820000000006985
-4.770000000000437
-0.26099999999996726
0.6980000000003201
-1.0
-16.180000000007567
8.160000000003492
-21.599999999998545
-10.81899999999996
-4.757999999999811
4.0
-31.139999999999418
0.8000000000029104
-68.89999999999418
-16.251999999999498
3.149999999999636
27.0
-109.14999999999418
39.5
-137.1999999999971
-5.022999999999229
28.152000000000044
47.0
-255.89999999999418
144.39999999999418
-214.45999999999913
3.320000000001528
50.314000000000306
68.0
-417.1000000000058
294.79999999998836
-306.1000000000058
7.120000000002619
60.73999999999978
109.0
-550.3999999999942
468.19999999995343
-413.4799999999959
13.270000000004075
63.63999999999942
162.0
-659.2999999999884
660.1000000000349
-545.3000000000029
12.760000000002037
67.68999999999869
222.0
-767.4000000000233
873.4000000000233
-710.8000000000029
8.87999999999738
70.90999999999985
285.0
-884.8000000000175
1117.1999999999534
-899.0
18.320000000006985
74.41699999999946
349.0
-1018.8000000000175
1447.0999999999767
-1106.1000000000058
36.669999999998254
81.96799999999985
442.0
-1196.5999999999913
1891.9000000000233
-1346.5
70.22999999999593
102.70400000000063
565.0
-1418.0999999999913
2464.600000000093
-1607.8999999999942
111.59000000000378
118.64400000000023
707.0
-1653.5
3175.4000000000233
-1917.1000000000058
153.9499999999971
129.08399999999983
915.0
-1925.1999999999825
4077.0
-2296.899999999994
241.59999999999854
166.67999999999847
1179.0
-2295.100000000006
5281.400000000023
-2749.7999999999884
375.5
216.47999999999956
1455.0
-2743.7999999999884
6810.700000000012
-3328.0
526.6699999999983
264.1800000000003
1806.0
-3206.2999999999884
8583.100000000035
-3991.6999999999825
692.5200000000004
301.52549999999997
2219.0
-3612.899999999994
10467.199999999953
-4662.100000000006
843.1419999999998
309.9320000000007
2661.0
-3891.770000000004
12299.299999999988
-5374.600000000006
940.2169999999999
284.6289999999999
3154.0
-4074.5
13923.299999999988
-6160.600000000006
997.8425
262.9546
3631.0
-4300.299999999988
15289.900000000023
-7025.5
1025.811
284.60000000000036
4059.0
-4683.5
16461.900000000023
-7964.800000000047
1033.7979999999998
329.9200000000019
4442.0
-5175.599999999977
17469.29999999999
-8891.700000000012
1020.6669999999999
344.75
4810.0
-5616.699999999953
18361.29999999999
-9809.900000000023
980.3070000000002
330.6900000000023
5254.0
-6014.0
19236.100000000035
-10787.799999999988
949.8160000000003
341.5900000000038
5702.0
-6518.399999999907
20228.5
-11812.199999999953
958.7497000000001
382.679999999993
6058.0
-7163.5
21570.5
-12924.20000000007
1032.9030000000002
418.59000000000015
6407.0
-7891.200000000012
23343.099999999977
-14214.699999999953
1154.300000000001
432.10999999999694
6789.0
-8696.899999999965
25473.300000000047
-15764.0
1285.0300000000025
471.3209999999999
7203.0
-9728.100000000006
27855.199999999953
-17619.899999999965
1417.7999999999993
581.5099999999998
7654.0
-11129.199999999983
30303.70000000007
-19662.900000000023
1537.1199999999953
739.362000000001
8032.0
-12793.699999999983
32741.400000000023
-21746.199999999953
1624.0899999999965
881.3289999999997
8290.0
-14495.400000000009
35095.19999999995
-23879.29999999999
1692.9599999999991
999.482
8541.0
-16133.600000000006
37327.90000000002
-26058.70000000001
1753.9400000000023
1127.1131
8819.0
-17668.0
39506.0
-28244.20000000007
1789.0199999999895
1235.2670000000007
9088.0
-18904.5
41558.0
-30373.300000000047
1782.0800000000017
1236.130000000001
9345.0
-19571.400000000023
43514.0
-32401.099999999977
1778.770000000004
1109.4799999999996
9536.0
-19656.5
45575.0
-34430.0
1818.8399999999965
948.2900000000009
9597.0
-19510.0
47828.0
-36567.80000000005
1862.9400000000096
825.3600000000006
9552.0
-19587.5
50341.0
-38816.09999999998
1944.510000000002
789.3199999999997
9445.0
-20201.0
53148.0
-41214.09999999998

So there is some significant difference which appears to grow with time, the time is 0.1 seconds at 7.2RPM, so that is  about 1% of a revolution. Will need to use the validation data to get the full answer.

**Oct 5 2020**

Working on getting the translation bugs out.  Here is what I get with the julia AC with reversed forces (since cactus spins clockwise):

FReactionHist 2 : 1 OLD: -7.142141e+02 NEW: 1.164421e+02
       FReactionHist 2 : 2 OLD: -1.076061e+02 NEW: 1.096553e+02
       FReactionHist 2 : 3 OLD: 1.515958e+05 NEW: 1.496342e+05
       FReactionHist 2 : 4 OLD: 7.069546e+01 NEW: 2.195384e+02
       FReactionHist 2 : 5 OLD: -7.420892e+02 NEW: 5.455120e+01
       FReactionHist 2 : 6 OLD: 1.359528e+02 NEW: -2.136797e+01
       FReactionHist 3 : 1 OLD: -3.359312e+03 NEW: 4.332666e+02
       FReactionHist 3 : 2 OLD: -4.588144e+02 NEW: 1.430888e+02
       FReactionHist 3 : 3 OLD: 4.326783e+05 NEW: 4.204167e+05
       FReactionHist 3 : 4 OLD: -1.020519e+03 NEW: 2.958482e+03
       FReactionHist 3 : 5 OLD: 3.280347e+03 NEW: -1.351769e+03
       FReactionHist 3 : 6 OLD: 1.258519e+03 NEW: -2.307564e+01
       FReactionHist 4 : 1 OLD: -3.979357e+03 NEW: -1.378124e+02
       FReactionHist 4 : 2 OLD: -1.340640e+02 NEW: -1.028013e+03
       FReactionHist 4 : 3 OLD: 7.319887e+05 NEW: 7.042254e+05
       FReactionHist 4 : 4 OLD: -6.307394e+03 NEW: 1.036015e+04
       FReactionHist 4 : 5 OLD: 2.314380e+04 NEW: -6.606696e+03
       FReactionHist 4 : 6 OLD: 3.744007e+03 NEW: 1.125207e+03
       FReactionHist 5 : 1 OLD: -1.750950e+03 NEW: -1.262099e+03
       FReactionHist 5 : 2 OLD: 1.599643e+03 NEW: -2.698553e+03
       FReactionHist 5 : 3 OLD: 9.992354e+05 NEW: 9.659360e+05
       FReactionHist 5 : 4 OLD: -1.744892e+04 NEW: 2.005435e+04
       FReactionHist 5 : 5 OLD: 6.147387e+04 NEW: -1.409653e+04
       FReactionHist 5 : 6 OLD: 3.374528e+03 NEW: 5.317006e+03
       FReactionHist 6 : 1 OLD: 2.905970e+03 NEW: -1.675415e+03
       FReactionHist 6 : 2 OLD: 4.352465e+03 NEW: -3.639308e+03
       FReactionHist 6 : 3 OLD: 1.273110e+06 NEW: 1.236127e+06
       FReactionHist 6 : 4 OLD: -3.212057e+04 NEW: 2.852020e+04
       FReactionHist 6 : 5 OLD: 1.179283e+05 NEW: -2.164084e+04
       FReactionHist 6 : 6 OLD: -2.245956e+03 NEW: 9.036582e+03
       FReactionHist 7 : 1 OLD: 9.451359e+03 NEW: -1.968974e+03
       FReactionHist 7 : 2 OLD: 6.298100e+03 NEW: -4.312284e+03
       FReactionHist 7 : 3 OLD: 1.535036e+06 NEW: 1.490311e+06
       FReactionHist 7 : 4 OLD: -4.252528e+04 NEW: 3.438311e+04
       FReactionHist 7 : 5 OLD: 1.766629e+05 NEW: -2.820633e+04
       FReactionHist 7 : 6 OLD: -9.761377e+03 NEW: 8.942055e+03
       FReactionHist 8 : 1 OLD: 7.140729e+03 NEW: -2.066443e+03
       FReactionHist 8 : 2 OLD: 5.190312e+03 NEW: -4.157384e+03
       FReactionHist 8 : 3 OLD: 1.800449e+06 NEW: 1.744631e+06
       FReactionHist 8 : 4 OLD: -4.293471e+04 NEW: 3.560894e+04
       FReactionHist 8 : 5 OLD: 2.083231e+05 NEW: -3.378951e+04
       FReactionHist 8 : 6 OLD: -1.689814e+04 NEW: 8.874783e+03
       FReactionHist 9 : 1 OLD: -5.777089e+03 NEW: -1.744955e+03
       FReactionHist 9 : 2 OLD: 3.081757e+03 NEW: -2.795530e+03
       FReactionHist 9 : 3 OLD: 2.059662e+06 NEW: 1.983068e+06
       FReactionHist 9 : 4 OLD: -4.110436e+04 NEW: 3.350876e+04
       FReactionHist 9 : 5 OLD: 2.195080e+05 NEW: -3.844403e+04
       FReactionHist 9 : 6 OLD: -2.325660e+04 NEW: 1.155808e+04
       FReactionHist 10 : 1 OLD: -1.532398e+04 NEW: -1.188792e+03
       FReactionHist 10 : 2 OLD: 4.241030e+03 NEW: -1.831360e+03
       FReactionHist 10 : 3 OLD: 2.292043e+06 NEW: 2.199802e+06
       FReactionHist 10 : 4 OLD: -4.820738e+04 NEW: 3.292204e+04
       FReactionHist 10 : 5 OLD: 2.564952e+05 NEW: -4.199670e+04
       FReactionHist 10 : 6 OLD: -2.816460e+04 NEW: 1.614135e+04
       FReactionHist 11 : 1 OLD: -8.625607e+03 NEW: -7.121494e+02
       FReactionHist 11 : 2 OLD: 7.871172e+03 NEW: -2.347302e+03
       FReactionHist 11 : 3 OLD: 2.496174e+06 NEW: 2.402529e+06
       FReactionHist 11 : 4 OLD: -6.118161e+04 NEW: 3.592116e+04
       FReactionHist 11 : 5 OLD: 3.693550e+05 NEW: -4.788219e+04
       FReactionHist 11 : 6 OLD: -3.092693e+04 NEW: 2.095585e+04
       FReactionHist 12 : 1 OLD: 1.378745e+04 NEW: -1.119941e+03
       FReactionHist 12 : 2 OLD: 8.676335e+03 NEW: -2.784092e+03
       FReactionHist 12 : 3 OLD: 2.661933e+06 NEW: 2.578619e+06
       FReactionHist 12 : 4 OLD: -6.678975e+04 NEW: 4.168977e+04
       FReactionHist 12 : 5 OLD: 5.642306e+05 NEW: -5.955439e+04
       FReactionHist 12 : 6 OLD: -3.419050e+04 NEW: 2.545180e+04
       FReactionHist 13 : 1 OLD: 4.188317e+04 NEW: -2.914790e+03
       FReactionHist 13 : 2 OLD: 5.949264e+03 NEW: -3.162142e+03
       FReactionHist 13 : 3 OLD: 2.809592e+06 NEW: 2.744518e+06
       FReactionHist 13 : 4 OLD: -6.122785e+04 NEW: 4.785815e+04
       FReactionHist 13 : 5 OLD: 7.879436e+05 NEW: -7.874522e+04
       FReactionHist 13 : 6 OLD: -4.106554e+04 NEW: 3.019324e+04
       FReactionHist 14 : 1 OLD: 5.831440e+04 NEW: -4.845920e+03
       FReactionHist 14 : 2 OLD: 3.728332e+03 NEW: -3.852956e+03
       FReactionHist 14 : 3 OLD: 2.946149e+06 NEW: 2.899501e+06
       FReactionHist 14 : 4 OLD: -5.653142e+04 NEW: 5.503728e+04
       FReactionHist 14 : 5 OLD: 9.839289e+05 NEW: -1.045748e+05
       FReactionHist 14 : 6 OLD: -5.029699e+04 NEW: 3.536937e+04
       FReactionHist 15 : 1 OLD: 6.152321e+04 NEW: -6.973072e+03
       FReactionHist 15 : 2 OLD: 4.844859e+03 NEW: -4.799935e+03
       FReactionHist 15 : 3 OLD: 3.057770e+06 NEW: 3.025861e+06
       FReactionHist 15 : 4 OLD: -6.372762e+04 NEW: 6.510324e+04
       FReactionHist 15 : 5 OLD: 1.143327e+06 NEW: -1.381536e+05
       FReactionHist 15 : 6 OLD: -6.014461e+04 NEW: 4.027162e+04
       FReactionHist 16 : 1 OLD: 6.731060e+04 NEW: -1.117982e+04
       FReactionHist 16 : 2 OLD: 8.376310e+03 NEW: -6.731109e+03
       FReactionHist 16 : 3 OLD: 3.153126e+06 NEW: 3.128519e+06
       FReactionHist 16 : 4 OLD: -7.872475e+04 NEW: 7.628414e+04
       FReactionHist 16 : 5 OLD: 1.305007e+06 NEW: -1.839333e+05
       FReactionHist 16 : 6 OLD: -7.497424e+04 NEW: 4.462945e+04
       FReactionHist 17 : 1 OLD: 8.230119e+04 NEW: -1.760902e+04
       FReactionHist 17 : 2 OLD: 1.024532e+04 NEW: -8.227951e+03
       FReactionHist 17 : 3 OLD: 3.230910e+06 NEW: 3.202602e+06
       FReactionHist 17 : 4 OLD: -8.822670e+04 NEW: 8.487581e+04
       FReactionHist 17 : 5 OLD: 1.510418e+06 NEW: -2.413729e+05
       FReactionHist 17 : 6 OLD: -9.765531e+04 NEW: 4.906374e+04
       FReactionHist 18 : 1 OLD: 1.065081e+05 NEW: -2.459696e+04
       FReactionHist 18 : 2 OLD: 7.938057e+03 NEW: -7.847645e+03
       FReactionHist 18 : 3 OLD: 3.273443e+06 NEW: 3.238778e+06
       FReactionHist 18 : 4 OLD: -8.456955e+04 NEW: 8.670658e+04
       FReactionHist 18 : 5 OLD: 1.764005e+06 NEW: -3.025647e+05
       FReactionHist 18 : 6 OLD: -1.227681e+05 NEW: 5.751900e+04
       FReactionHist 19 : 1 OLD: 1.394250e+05 NEW: -3.036548e+04
       FReactionHist 19 : 2 OLD: 4.080830e+03 NEW: -6.248048e+03
       FReactionHist 19 : 3 OLD: 3.285289e+06 NEW: 3.248331e+06
       FReactionHist 19 : 4 OLD: -7.720880e+04 NEW: 8.238303e+04
       FReactionHist 19 : 5 OLD: 2.048196e+06 NEW: -3.581247e+05
       FReactionHist 19 : 6 OLD: -1.418991e+05 NEW: 7.360885e+04
       FReactionHist 20 : 1 OLD: 1.754878e+05 NEW: -3.437892e+04
       FReactionHist 20 : 2 OLD: 4.148639e+03 NEW: -4.969767e+03
       FReactionHist 20 : 3 OLD: 3.268342e+06 NEW: 3.233945e+06
       FReactionHist 20 : 4 OLD: -8.418677e+04 NEW: 7.887574e+04
       FReactionHist 20 : 5 OLD: 2.308836e+06 NEW: -4.011798e+05
       FReactionHist 20 : 6 OLD: -1.498487e+05 NEW: 9.505822e+04
       FReactionHist 21 : 1 OLD: 1.920116e+05 NEW: -3.630932e+04
       FReactionHist 21 : 2 OLD: 9.263111e+03 NEW: -4.679147e+03
       FReactionHist 21 : 3 OLD: 3.214659e+06 NEW: 3.184487e+06
       FReactionHist 21 : 4 OLD: -1.081526e+05 NEW: 7.836634e+04
       FReactionHist 21 : 5 OLD: 2.448197e+06 NEW: -4.277050e+05
       FReactionHist 21 : 6 OLD: -1.464631e+05 NEW: 1.159468e+05
       FReactionHist 22 : 1 OLD: 1.718877e+05 NEW: -3.606751e+04
       FReactionHist 22 : 2 OLD: 1.438798e+04 NEW: -4.738759e+03
       FReactionHist 22 : 3 OLD: 3.118671e+06 NEW: 3.096135e+06
       FReactionHist 22 : 4 OLD: -1.320333e+05 NEW: 7.967334e+04
       FReactionHist 22 : 5 OLD: 2.423031e+06 NEW: -4.352086e+05
       FReactionHist 22 : 6 OLD: -1.365775e+05 NEW: 1.308296e+05
       FReactionHist 23 : 1 OLD: 1.338156e+05 NEW: -3.316949e+04
       FReactionHist 23 : 2 OLD: 1.428200e+04 NEW: -4.310009e+03
       FReactionHist 23 : 3 OLD: 2.979634e+06 NEW: 2.969742e+06
       FReactionHist 23 : 4 OLD: -1.393290e+05 NEW: 8.240883e+04
       FReactionHist 23 : 5 OLD: 2.306570e+06 NEW: -4.230741e+05
       FReactionHist 23 : 6 OLD: -1.308751e+05 NEW: 1.390640e+05
       FReactionHist 24 : 1 OLD: 1.124152e+05 NEW: -2.807951e+04
       FReactionHist 24 : 2 OLD: 1.084510e+04 NEW: -4.623080e+03
       FReactionHist 24 : 3 OLD: 2.817431e+06 NEW: 2.824350e+06
       FReactionHist 24 : 4 OLD: -1.336725e+05 NEW: 8.685459e+04
       FReactionHist 24 : 5 OLD: 2.216584e+06 NEW: -3.944644e+05
       FReactionHist 24 : 6 OLD: -1.385456e+05 NEW: 1.421783e+05
       FReactionHist 25 : 1 OLD: 1.074521e+05 NEW: -2.182876e+04
       FReactionHist 25 : 2 OLD: 9.615636e+03 NEW: -5.828846e+03
       FReactionHist 25 : 3 OLD: 2.657899e+06 NEW: 2.688181e+06
       FReactionHist 25 : 4 OLD: -1.309875e+05 NEW: 9.239355e+04
       FReactionHist 25 : 5 OLD: 2.190722e+06 NEW: -3.559603e+05
       FReactionHist 25 : 6 OLD: -1.564494e+05 NEW: 1.404112e+05
       FReactionHist 26 : 1 OLD: 9.848444e+04 NEW: -1.548123e+04
       FReactionHist 26 : 2 OLD: 1.120493e+04 NEW: -5.825256e+03
       FReactionHist 26 : 3 OLD: 2.527702e+06 NEW: 2.577742e+06
       FReactionHist 26 : 4 OLD: -1.361058e+05 NEW: 9.258007e+04
       FReactionHist 26 : 5 OLD: 2.196934e+06 NEW: -3.149337e+05
       FReactionHist 26 : 6 OLD: -1.715531e+05 NEW: 1.361151e+05
       FReactionHist 27 : 1 OLD: 9.141744e+04 NEW: -9.890415e+03
       FReactionHist 27 : 2 OLD: 1.228409e+04 NEW: -3.244756e+03
       FReactionHist 27 : 3 OLD: 2.462574e+06 NEW: 2.515542e+06
       FReactionHist 27 : 4 OLD: -1.417384e+05 NEW: 8.392503e+04
       FReactionHist 27 : 5 OLD: 2.244657e+06 NEW: -2.779866e+05
       FReactionHist 27 : 6 OLD: -1.768987e+05 NEW: 1.349395e+05
       FReactionHist 28 : 1 OLD: 1.038709e+05 NEW: -5.551201e+03
       FReactionHist 28 : 2 OLD: 1.210098e+04 NEW: 1.734187e+02
       FReactionHist 28 : 3 OLD: 2.444617e+06 NEW: 2.492861e+06
       FReactionHist 28 : 4 OLD: -1.425837e+05 NEW: 7.407420e+04
       FReactionHist 28 : 5 OLD: 2.359731e+06 NEW: -2.505231e+05
       FReactionHist 28 : 6 OLD: -1.750634e+05 NEW: 1.408323e+05
       FReactionHist 29 : 1 OLD: 1.249837e+05 NEW: -3.404476e+03
       FReactionHist 29 : 2 OLD: 1.114729e+04 NEW: 1.842455e+03
       FReactionHist 29 : 3 OLD: 2.393481e+06 NEW: 2.446834e+06
       FReactionHist 29 : 4 OLD: -1.380475e+05 NEW: 7.292314e+04
       FReactionHist 29 : 5 OLD: 2.501623e+06 NEW: -2.354972e+05
       FReactionHist 29 : 6 OLD: -1.721517e+05 NEW: 1.560046e+05
       FReactionHist 30 : 1 OLD: 1.322929e+05 NEW: -2.953173e+03
       FReactionHist 30 : 2 OLD: 9.029000e+03 NEW: 1.478806e+03
       FReactionHist 30 : 3 OLD: 2.263661e+06 NEW: 2.323826e+06
       FReactionHist 30 : 4 OLD: -1.313334e+05 NEW: 8.686989e+04
       FReactionHist 30 : 5 OLD: 2.599713e+06 NEW: -2.302805e+05
       FReactionHist 30 : 6 OLD: -1.753254e+05 NEW: 1.788298e+05
       FReactionHist 31 : 1 OLD: 1.269810e+05 NEW: -2.122427e+03
       FReactionHist 31 : 2 OLD: 8.251867e+03 NEW: -1.687188e+03
       FReactionHist 31 : 3 OLD: 2.069228e+06 NEW: 2.126314e+06
       FReactionHist 31 : 4 OLD: -1.302810e+05 NEW: 1.195187e+05
       FReactionHist 31 : 5 OLD: 2.630570e+06 NEW: -2.277923e+05
       FReactionHist 31 : 6 OLD: -1.923637e+05 NEW: 2.036447e+05
       FReactionHist 32 : 1 OLD: 1.167076e+05 NEW: -1.878170e+02
       FReactionHist 32 : 2 OLD: 1.051775e+04 NEW: -7.806415e+03
       FReactionHist 32 : 3 OLD: 1.866105e+06 NEW: 1.916800e+06
       FReactionHist 32 : 4 OLD: -1.376543e+05 NEW: 1.676884e+05
       FReactionHist 32 : 5 OLD: 2.619023e+06 NEW: -2.259328e+05
       FReactionHist 32 : 6 OLD: -2.250440e+05 NEW: 2.250962e+05
       FReactionHist 33 : 1 OLD: 1.055757e+05 NEW: 5.932327e+02
       FReactionHist 33 : 2 OLD: 1.276755e+04 NEW: -1.491795e+04
       FReactionHist 33 : 3 OLD: 1.707179e+06 NEW: 1.743536e+06
       FReactionHist 33 : 4 OLD: -1.436184e+05 NEW: 2.224612e+05
       FReactionHist 33 : 5 OLD: 2.593474e+06 NEW: -2.284145e+05
       FReactionHist 33 : 6 OLD: -2.657375e+05 NEW: 2.422050e+05
       FReactionHist 34 : 1 OLD: 9.385162e+04 NEW: 2.739296e+02
       FReactionHist 34 : 2 OLD: 1.134134e+04 NEW: -2.102680e+04
       FReactionHist 34 : 3 OLD: 1.575011e+06 NEW: 1.586048e+06
       FReactionHist 34 : 4 OLD: -1.368330e+05 NEW: 2.729801e+05
       FReactionHist 34 : 5 OLD: 2.576019e+06 NEW: -2.358331e+05
       FReactionHist 34 : 6 OLD: -3.041237e+05 NEW: 2.587387e+05
       FReactionHist 35 : 1 OLD: 8.637810e+04 NEW: 8.409079e+02
       FReactionHist 35 : 2 OLD: 6.853035e+03 NEW: -2.574276e+04
       FReactionHist 35 : 3 OLD: 1.411005e+06 NEW: 1.412792e+06
       FReactionHist 35 : 4 OLD: -1.185265e+05 NEW: 3.096791e+05
       FReactionHist 35 : 5 OLD: 2.604190e+06 NEW: -2.461698e+05
       FReactionHist 35 : 6 OLD: -3.320457e+05 NEW: 2.763023e+05
       FReactionHist 36 : 1 OLD: 9.490298e+04 NEW: 9.706199e+02
       FReactionHist 36 : 2 OLD: 2.280761e+03 NEW: -2.744796e+04
       FReactionHist 36 : 3 OLD: 1.238541e+06 NEW: 1.239934e+06
       FReactionHist 36 : 4 OLD: -1.021918e+05 NEW: 3.245177e+05
       FReactionHist 36 : 5 OLD: 2.724584e+06 NEW: -2.639796e+05
       FReactionHist 36 : 6 OLD: -3.432683e+05 NEW: 2.933150e+05
       FReactionHist 37 : 1 OLD: 1.268966e+05 NEW: -1.989556e+03
       FReactionHist 37 : 2 OLD: 1.078024e+03 NEW: -2.480369e+04
       FReactionHist 37 : 3 OLD: 1.135310e+06 NEW: 1.117865e+06
       FReactionHist 37 : 4 OLD: -1.033049e+05 NEW: 3.135590e+05
       FReactionHist 37 : 5 OLD: 2.944936e+06 NEW: -2.944466e+05
       FReactionHist 37 : 6 OLD: -3.360152e+05 NEW: 3.133014e+05
       FReactionHist 38 : 1 OLD: 1.668647e+05 NEW: -6.098434e+03
       FReactionHist 38 : 2 OLD: 4.840742e+03 NEW: -1.929814e+04
       FReactionHist 38 : 3 OLD: 1.110722e+06 NEW: 1.063423e+06
       FReactionHist 38 : 4 OLD: -1.257647e+05 NEW: 2.811819e+05
       FReactionHist 38 : 5 OLD: 3.191046e+06 NEW: -3.337491e+05
       FReactionHist 38 : 6 OLD: -3.134078e+05 NEW: 3.426309e+05
       FReactionHist 39 : 1 OLD: 1.856342e+05 NEW: -9.131407e+03
       FReactionHist 39 : 2 OLD: 1.001564e+04 NEW: -1.284482e+04
       FReactionHist 39 : 3 OLD: 1.133153e+06 NEW: 1.043655e+06
       FReactionHist 39 : 4 OLD: -1.569109e+05 NEW: 2.381412e+05
       FReactionHist 39 : 5 OLD: 3.362434e+06 NEW: -3.761506e+05
       FReactionHist 39 : 6 OLD: -2.778774e+05 NEW: 3.798467e+05
       FReactionHist 40 : 1 OLD: 1.800385e+05 NEW: -1.280064e+04
       FReactionHist 40 : 2 OLD: 1.141842e+04 NEW: -6.471953e+03
       FReactionHist 40 : 3 OLD: 1.211304e+06 NEW: 1.059864e+06
       FReactionHist 40 : 4 OLD: -1.805170e+05 NEW: 1.934087e+05
       FReactionHist 40 : 5 OLD: 3.441220e+06 NEW: -4.219744e+05
       FReactionHist 40 : 6 OLD: -2.376497e+05 NEW: 4.155162e+05
       FReactionHist 41 : 1 OLD: 1.760826e+05 NEW: -1.731452e+04
       FReactionHist 41 : 2 OLD: 9.209623e+03 NEW: -9.916724e+02
       FReactionHist 41 : 3 OLD: 1.361596e+06 NEW: 1.156731e+06
       FReactionHist 41 : 4 OLD: -1.971051e+05 NEW: 1.551130e+05
       FReactionHist 41 : 5 OLD: 3.502013e+06 NEW: -4.720036e+05
       FReactionHist 41 : 6 OLD: -2.110962e+05 NEW: 4.378619e+05
       FReactionHist 42 : 1 OLD: 1.915293e+05 NEW: -2.157511e+04
       FReactionHist 42 : 2 OLD: 9.189790e+03 NEW: 2.216717e+03
       FReactionHist 42 : 3 OLD: 1.564778e+06 NEW: 1.327068e+06
       FReactionHist 42 : 4 OLD: -2.244041e+05 NEW: 1.317321e+05
       FReactionHist 42 : 5 OLD: 3.603496e+06 NEW: -5.245780e+05
       FReactionHist 42 : 6 OLD: -2.116156e+05 NEW: 4.452796e+05
       FReactionHist 43 : 1 OLD: 2.134499e+05 NEW: -2.632113e+04
       FReactionHist 43 : 2 OLD: 1.493455e+04 NEW: 2.573444e+03
       FReactionHist 43 : 3 OLD: 1.747241e+06 NEW: 1.490436e+06
       FReactionHist 43 : 4 OLD: -2.725513e+05 NEW: 1.266104e+05
       FReactionHist 43 : 5 OLD: 3.718355e+06 NEW: -5.793222e+05
       FReactionHist 43 : 6 OLD: -2.328435e+05 NEW: 4.498338e+05
       FReactionHist 44 : 1 OLD: 2.279972e+05 NEW: -3.199499e+04
       FReactionHist 44 : 2 OLD: 2.318336e+04 NEW: 6.030375e+02
       FReactionHist 44 : 3 OLD: 1.852176e+06 NEW: 1.599582e+06
       FReactionHist 44 : 4 OLD: -3.291470e+05 NEW: 1.389689e+05
       FReactionHist 44 : 5 OLD: 3.794033e+06 NEW: -6.350240e+05
       FReactionHist 44 : 6 OLD: -2.581807e+05 NEW: 4.618161e+05
       FReactionHist 45 : 1 OLD: 2.346854e+05 NEW: -3.654262e+04
       FReactionHist 45 : 2 OLD: 2.748708e+04 NEW: -2.630075e+03
       FReactionHist 45 : 3 OLD: 1.926817e+06 NEW: 1.694493e+06
       FReactionHist 45 : 4 OLD: -3.737321e+05 NEW: 1.650359e+05
       FReactionHist 45 : 5 OLD: 3.795662e+06 NEW: -6.863294e+05
       FReactionHist 45 : 6 OLD: -2.786692e+05 NEW: 4.794858e+05
       FReactionHist 46 : 1 OLD: 2.270696e+05 NEW: -4.009947e+04
       FReactionHist 46 : 2 OLD: 2.728351e+04 NEW: -6.873510e+03
       FReactionHist 46 : 3 OLD: 2.027003e+06 NEW: 1.814811e+06
       FReactionHist 46 : 4 OLD: -4.032465e+05 NEW: 1.993312e+05
       FReactionHist 46 : 5 OLD: 3.693849e+06 NEW: -7.304068e+05
       FReactionHist 46 : 6 OLD: -3.032877e+05 NEW: 5.000955e+05
       FReactionHist 47 : 1 OLD: 1.968345e+05 NEW: -4.325731e+04
       FReactionHist 47 : 2 OLD: 2.843782e+04 NEW: -1.107327e+04
       FReactionHist 47 : 3 OLD: 2.136777e+06 NEW: 1.942718e+06
       FReactionHist 47 : 4 OLD: -4.331840e+05 NEW: 2.361186e+05
       FReactionHist 47 : 5 OLD: 3.495431e+06 NEW: -7.642749e+05
       FReactionHist 47 : 6 OLD: -3.480500e+05 NEW: 5.274939e+05
       FReactionHist 48 : 1 OLD: 1.658532e+05 NEW: -4.378409e+04
       FReactionHist 48 : 2 OLD: 3.391308e+04 NEW: -1.385201e+04
       FReactionHist 48 : 3 OLD: 2.234898e+06 NEW: 2.057571e+06
       FReactionHist 48 : 4 OLD: -4.698479e+05 NEW: 2.680328e+05
       FReactionHist 48 : 5 OLD: 3.270410e+06 NEW: -7.821996e+05
       FReactionHist 48 : 6 OLD: -4.089249e+05 NEW: 5.632436e+05
       FReactionHist 49 : 1 OLD: 1.534348e+05 NEW: -4.160078e+04
       FReactionHist 49 : 2 OLD: 3.852997e+04 NEW: -1.453489e+04
       FReactionHist 49 : 3 OLD: 2.328983e+06 NEW: 2.176630e+06
       FReactionHist 49 : 4 OLD: -4.978654e+05 NEW: 2.917929e+05
       FReactionHist 49 : 5 OLD: 3.081498e+06 NEW: -7.843108e+05
       FReactionHist 49 : 6 OLD: -4.587724e+05 NEW: 6.015549e+05
       FReactionHist 50 : 1 OLD: 1.448550e+05 NEW: -3.924307e+04
       FReactionHist 50 : 2 OLD: 3.742459e+04 NEW: -1.422422e+04
       FReactionHist 50 : 3 OLD: 2.429362e+06 NEW: 2.311985e+06
       FReactionHist 50 : 4 OLD: -5.026903e+05 NEW: 3.118249e+05
       FReactionHist 50 : 5 OLD: 2.915128e+06 NEW: -7.744515e+05
       FReactionHist 50 : 6 OLD: -4.778675e+05 NEW: 6.395634e+05
       FReactionHist 51 : 1 OLD: 1.276172e+05 NEW: -3.551463e+04
       FReactionHist 51 : 2 OLD: 3.283271e+04 NEW: -1.487836e+04
       FReactionHist 51 : 3 OLD: 2.543811e+06 NEW: 2.457742e+06
       FReactionHist 51 : 4 OLD: -4.905020e+05 NEW: 3.336533e+05
       FReactionHist 51 : 5 OLD: 2.737870e+06 NEW: -7.533141e+05
       FReactionHist 51 : 6 OLD: -4.717114e+05 NEW: 6.789870e+05
       
Now we remove the negative on the tangential force and rerun and its basically the same...:

       FReactionHist 2 : 1 OLD: -7.142141e+02 NEW: 1.172042e+02
       FReactionHist 2 : 2 OLD: -1.076061e+02 NEW: 1.093825e+02
       FReactionHist 2 : 3 OLD: 1.515958e+05 NEW: 1.496260e+05
       FReactionHist 2 : 4 OLD: 7.069546e+01 NEW: 2.311622e+02
       FReactionHist 2 : 5 OLD: -7.420892e+02 NEW: 4.707559e+01
       FReactionHist 2 : 6 OLD: 1.359528e+02 NEW: -3.199672e+01
       FReactionHist 3 : 1 OLD: -3.359312e+03 NEW: 4.187249e+02
       FReactionHist 3 : 2 OLD: -4.588144e+02 NEW: 1.238238e+02
       FReactionHist 3 : 3 OLD: 4.326783e+05 NEW: 4.203771e+05
       FReactionHist 3 : 4 OLD: -1.020519e+03 NEW: 3.026332e+03
       FReactionHist 3 : 5 OLD: 3.280347e+03 NEW: -1.438040e+03
       FReactionHist 3 : 6 OLD: 1.258519e+03 NEW: -1.385523e+02
       FReactionHist 4 : 1 OLD: -3.979357e+03 NEW: -2.278271e+02
       FReactionHist 4 : 2 OLD: -1.340640e+02 NEW: -1.087514e+03
       FReactionHist 4 : 3 OLD: 7.319887e+05 NEW: 7.041916e+05
       FReactionHist 4 : 4 OLD: -6.307394e+03 NEW: 1.038866e+04
       FReactionHist 4 : 5 OLD: 2.314380e+04 NEW: -6.811097e+03
       FReactionHist 4 : 6 OLD: 3.744007e+03 NEW: 6.481722e+02
       FReactionHist 5 : 1 OLD: -1.750950e+03 NEW: -1.391177e+03
       FReactionHist 5 : 2 OLD: 1.599643e+03 NEW: -2.705260e+03
       FReactionHist 5 : 3 OLD: 9.992354e+05 NEW: 9.660089e+05
       FReactionHist 5 : 4 OLD: -1.744892e+04 NEW: 1.966659e+04
       FReactionHist 5 : 5 OLD: 6.147387e+04 NEW: -1.414200e+04
       FReactionHist 5 : 6 OLD: 3.374528e+03 NEW: 4.296711e+03
       FReactionHist 6 : 1 OLD: 2.905970e+03 NEW: -1.721809e+03
       FReactionHist 6 : 2 OLD: 4.352465e+03 NEW: -3.488495e+03
       FReactionHist 6 : 3 OLD: 1.273110e+06 NEW: 1.236258e+06
       FReactionHist 6 : 4 OLD: -3.212057e+04 NEW: 2.741805e+04
       FReactionHist 6 : 5 OLD: 1.179283e+05 NEW: -2.115668e+04
       FReactionHist 6 : 6 OLD: -2.245956e+03 NEW: 7.583230e+03
       FReactionHist 7 : 1 OLD: 9.451359e+03 NEW: -1.967471e+03
       FReactionHist 7 : 2 OLD: 6.298100e+03 NEW: -4.073436e+03
       FReactionHist 7 : 3 OLD: 1.535036e+06 NEW: 1.490478e+06
       FReactionHist 7 : 4 OLD: -4.252528e+04 NEW: 3.278086e+04
       FReactionHist 7 : 5 OLD: 1.766629e+05 NEW: -2.708201e+04
       FReactionHist 7 : 6 OLD: -9.761377e+03 NEW: 7.249986e+03
       FReactionHist 8 : 1 OLD: 7.140729e+03 NEW: -2.091399e+03
       FReactionHist 8 : 2 OLD: 5.190312e+03 NEW: -3.984269e+03
       FReactionHist 8 : 3 OLD: 1.800449e+06 NEW: 1.745004e+06
       FReactionHist 8 : 4 OLD: -4.293471e+04 NEW: 3.405181e+04
       FReactionHist 8 : 5 OLD: 2.083231e+05 NEW: -3.212116e+04
       FReactionHist 8 : 6 OLD: -1.689814e+04 NEW: 6.941767e+03
       FReactionHist 9 : 1 OLD: -5.777089e+03 NEW: -1.788985e+03
       FReactionHist 9 : 2 OLD: 3.081757e+03 NEW: -2.768798e+03
       FReactionHist 9 : 3 OLD: 2.059662e+06 NEW: 1.983733e+06
       FReactionHist 9 : 4 OLD: -4.110436e+04 NEW: 3.226863e+04
       FReactionHist 9 : 5 OLD: 2.195080e+05 NEW: -3.632106e+04
       FReactionHist 9 : 6 OLD: -2.325660e+04 NEW: 9.123991e+03
       FReactionHist 10 : 1 OLD: -1.532398e+04 NEW: -1.288776e+03
       FReactionHist 10 : 2 OLD: 4.241030e+03 NEW: -1.862991e+03
       FReactionHist 10 : 3 OLD: 2.292043e+06 NEW: 2.200733e+06
       FReactionHist 10 : 4 OLD: -4.820738e+04 NEW: 3.168880e+04
       FReactionHist 10 : 5 OLD: 2.564952e+05 NEW: -3.937167e+04
       FReactionHist 10 : 6 OLD: -2.816460e+04 NEW: 1.295142e+04
       FReactionHist 11 : 1 OLD: -8.625607e+03 NEW: -8.617217e+02
       FReactionHist 11 : 2 OLD: 7.871172e+03 NEW: -2.300872e+03
       FReactionHist 11 : 3 OLD: 2.496174e+06 NEW: 2.403598e+06
       FReactionHist 11 : 4 OLD: -6.118161e+04 NEW: 3.418035e+04
       FReactionHist 11 : 5 OLD: 3.693550e+05 NEW: -4.450729e+04
       FReactionHist 11 : 6 OLD: -3.092693e+04 NEW: 1.701199e+04
       FReactionHist 12 : 1 OLD: 1.378745e+04 NEW: -1.159627e+03
       FReactionHist 12 : 2 OLD: 8.676335e+03 NEW: -2.632307e+03
       FReactionHist 12 : 3 OLD: 2.661933e+06 NEW: 2.579720e+06
       FReactionHist 12 : 4 OLD: -6.678975e+04 NEW: 3.921683e+04
       FReactionHist 12 : 5 OLD: 5.642306e+05 NEW: -5.475729e+04
       FReactionHist 12 : 6 OLD: -3.419050e+04 NEW: 2.081603e+04
       FReactionHist 13 : 1 OLD: 4.188317e+04 NEW: -2.742688e+03
       FReactionHist 13 : 2 OLD: 5.949264e+03 NEW: -2.963469e+03
       FReactionHist 13 : 3 OLD: 2.809592e+06 NEW: 2.745845e+06
       FReactionHist 13 : 4 OLD: -6.122785e+04 NEW: 4.473603e+04
       FReactionHist 13 : 5 OLD: 7.879436e+05 NEW: -7.195373e+04
       FReactionHist 13 : 6 OLD: -4.106554e+04 NEW: 2.477928e+04
       FReactionHist 14 : 1 OLD: 5.831440e+04 NEW: -4.544152e+03
       FReactionHist 14 : 2 OLD: 3.728332e+03 NEW: -3.641504e+03
       FReactionHist 14 : 3 OLD: 2.946149e+06 NEW: 2.901394e+06
       FReactionHist 14 : 4 OLD: -5.653142e+04 NEW: 5.135246e+04
       FReactionHist 14 : 5 OLD: 9.839289e+05 NEW: -9.568841e+04
       FReactionHist 14 : 6 OLD: -5.029699e+04 NEW: 2.910417e+04
       FReactionHist 15 : 1 OLD: 6.152321e+04 NEW: -6.587560e+03
       FReactionHist 15 : 2 OLD: 4.844859e+03 NEW: -4.533479e+03
       FReactionHist 15 : 3 OLD: 3.057770e+06 NEW: 3.028540e+06
       FReactionHist 15 : 4 OLD: -6.372762e+04 NEW: 6.068001e+04
       FReactionHist 15 : 5 OLD: 1.143327e+06 NEW: -1.267977e+05
       FReactionHist 15 : 6 OLD: -6.014461e+04 NEW: 3.307065e+04
       FReactionHist 16 : 1 OLD: 6.731060e+04 NEW: -1.053479e+04
       FReactionHist 16 : 2 OLD: 8.376310e+03 NEW: -6.322789e+03
       FReactionHist 16 : 3 OLD: 3.153126e+06 NEW: 3.132043e+06
       FReactionHist 16 : 4 OLD: -7.872475e+04 NEW: 7.089602e+04
       FReactionHist 16 : 5 OLD: 1.305007e+06 NEW: -1.692659e+05
       FReactionHist 16 : 6 OLD: -7.497424e+04 NEW: 3.635063e+04
       FReactionHist 17 : 1 OLD: 8.230119e+04 NEW: -1.652364e+04
       FReactionHist 17 : 2 OLD: 1.024532e+04 NEW: -7.686036e+03
       FReactionHist 17 : 3 OLD: 3.230910e+06 NEW: 3.206863e+06
       FReactionHist 17 : 4 OLD: -8.822670e+04 NEW: 7.863287e+04
       FReactionHist 17 : 5 OLD: 1.510418e+06 NEW: -2.224358e+05
       FReactionHist 17 : 6 OLD: -9.765531e+04 NEW: 3.953574e+04
       FReactionHist 18 : 1 OLD: 1.065081e+05 NEW: -2.300920e+04
       FReactionHist 18 : 2 OLD: 7.938057e+03 NEW: -7.312113e+03
       FReactionHist 18 : 3 OLD: 3.273443e+06 NEW: 3.243760e+06
       FReactionHist 18 : 4 OLD: -8.456955e+04 NEW: 8.006873e+04
       FReactionHist 18 : 5 OLD: 1.764005e+06 NEW: -2.787848e+05
       FReactionHist 18 : 6 OLD: -1.227681e+05 NEW: 4.644758e+04
       FReactionHist 19 : 1 OLD: 1.394250e+05 NEW: -2.827976e+04
       FReactionHist 19 : 2 OLD: 4.080830e+03 NEW: -5.814822e+03
       FReactionHist 19 : 3 OLD: 3.285289e+06 NEW: 3.254283e+06
       FReactionHist 19 : 4 OLD: -7.720880e+04 NEW: 7.572450e+04
       FReactionHist 19 : 5 OLD: 2.048196e+06 NEW: -3.297734e+05
       FReactionHist 19 : 6 OLD: -1.418991e+05 NEW: 6.075321e+04
       FReactionHist 20 : 1 OLD: 1.754878e+05 NEW: -3.196144e+04
       FReactionHist 20 : 2 OLD: 4.148639e+03 NEW: -4.598017e+03
       FReactionHist 20 : 3 OLD: 3.268342e+06 NEW: 3.241117e+06
       FReactionHist 20 : 4 OLD: -8.418677e+04 NEW: 7.207135e+04
       FReactionHist 20 : 5 OLD: 2.308836e+06 NEW: -3.691300e+05
       FReactionHist 20 : 6 OLD: -1.498487e+05 NEW: 8.048500e+04
       FReactionHist 21 : 1 OLD: 1.920116e+05 NEW: -3.375937e+04
       FReactionHist 21 : 2 OLD: 9.263111e+03 NEW: -4.243756e+03
       FReactionHist 21 : 3 OLD: 3.214659e+06 NEW: 3.192696e+06
       FReactionHist 21 : 4 OLD: -1.081526e+05 NEW: 7.107272e+04
       FReactionHist 21 : 5 OLD: 2.448197e+06 NEW: -3.933337e+05
       FReactionHist 21 : 6 OLD: -1.464631e+05 NEW: 9.999930e+04
       FReactionHist 22 : 1 OLD: 1.718877e+05 NEW: -3.355836e+04
       FReactionHist 22 : 2 OLD: 1.438798e+04 NEW: -4.246661e+03
       FReactionHist 22 : 3 OLD: 3.118671e+06 NEW: 3.104883e+06
       FReactionHist 22 : 4 OLD: -1.320333e+05 NEW: 7.189940e+04
       FReactionHist 22 : 5 OLD: 2.423031e+06 NEW: -4.000459e+05
       FReactionHist 22 : 6 OLD: -1.365775e+05 NEW: 1.139332e+05
       FReactionHist 23 : 1 OLD: 1.338156e+05 NEW: -3.088976e+04
       FReactionHist 23 : 2 OLD: 1.428200e+04 NEW: -3.882671e+03
       FReactionHist 23 : 3 OLD: 2.979634e+06 NEW: 2.978787e+06
       FReactionHist 23 : 4 OLD: -1.393290e+05 NEW: 7.435978e+04
       FReactionHist 23 : 5 OLD: 2.306570e+06 NEW: -3.884183e+05
       FReactionHist 23 : 6 OLD: -1.308751e+05 NEW: 1.213598e+05
       FReactionHist 24 : 1 OLD: 1.124152e+05 NEW: -2.615894e+04
       FReactionHist 24 : 2 OLD: 1.084510e+04 NEW: -4.228635e+03
       FReactionHist 24 : 3 OLD: 2.817431e+06 NEW: 2.833730e+06
       FReactionHist 24 : 4 OLD: -1.336725e+05 NEW: 7.849808e+04
       FReactionHist 24 : 5 OLD: 2.216584e+06 NEW: -3.613800e+05
       FReactionHist 24 : 6 OLD: -1.385456e+05 NEW: 1.234923e+05
       FReactionHist 25 : 1 OLD: 1.074521e+05 NEW: -2.031132e+04
       FReactionHist 25 : 2 OLD: 9.615636e+03 NEW: -5.351260e+03
       FReactionHist 25 : 3 OLD: 2.657899e+06 NEW: 2.697804e+06
       FReactionHist 25 : 4 OLD: -1.309875e+05 NEW: 8.360152e+04
       FReactionHist 25 : 5 OLD: 2.190722e+06 NEW: -3.248730e+05
       FReactionHist 25 : 6 OLD: -1.564494e+05 NEW: 1.207526e+05
       FReactionHist 26 : 1 OLD: 9.848444e+04 NEW: -1.432356e+04
       FReactionHist 26 : 2 OLD: 1.120493e+04 NEW: -5.357684e+03
       FReactionHist 26 : 3 OLD: 2.527702e+06 NEW: 2.587416e+06
       FReactionHist 26 : 4 OLD: -1.361058e+05 NEW: 8.367320e+04
       FReactionHist 26 : 5 OLD: 2.196934e+06 NEW: -2.857611e+05
       FReactionHist 26 : 6 OLD: -1.715531e+05 NEW: 1.157218e+05
       FReactionHist 27 : 1 OLD: 9.141744e+04 NEW: -9.035275e+03
       FReactionHist 27 : 2 OLD: 1.228409e+04 NEW: -2.987146e+03
       FReactionHist 27 : 3 OLD: 2.462574e+06 NEW: 2.525155e+06
       FReactionHist 27 : 4 OLD: -1.417384e+05 NEW: 7.548240e+04
       FReactionHist 27 : 5 OLD: 2.244657e+06 NEW: -2.504677e+05
       FReactionHist 27 : 6 OLD: -1.768987e+05 NEW: 1.139736e+05
       FReactionHist 28 : 1 OLD: 1.038709e+05 NEW: -4.963290e+03
       FReactionHist 28 : 2 OLD: 1.210098e+04 NEW: 1.988071e+02
       FReactionHist 28 : 3 OLD: 2.444617e+06 NEW: 2.502037e+06
       FReactionHist 28 : 4 OLD: -1.425837e+05 NEW: 6.602014e+04
       FReactionHist 28 : 5 OLD: 2.359731e+06 NEW: -2.241183e+05
       FReactionHist 28 : 6 OLD: -1.750634e+05 NEW: 1.192029e+05
       FReactionHist 29 : 1 OLD: 1.249837e+05 NEW: -2.954603e+03
       FReactionHist 29 : 2 OLD: 1.114729e+04 NEW: 1.793701e+03
       FReactionHist 29 : 3 OLD: 2.393481e+06 NEW: 2.455130e+06
       FReactionHist 29 : 4 OLD: -1.380475e+05 NEW: 6.439929e+04
       FReactionHist 29 : 5 OLD: 2.501623e+06 NEW: -2.095322e+05
       FReactionHist 29 : 6 OLD: -1.721517e+05 NEW: 1.333070e+05
       FReactionHist 30 : 1 OLD: 1.322929e+05 NEW: -2.521969e+03
       FReactionHist 30 : 2 OLD: 9.029000e+03 NEW: 1.536157e+03
       FReactionHist 30 : 3 OLD: 2.263661e+06 NEW: 2.331380e+06
       FReactionHist 30 : 4 OLD: -1.313334e+05 NEW: 7.664457e+04
       FReactionHist 30 : 5 OLD: 2.599713e+06 NEW: -2.042665e+05
       FReactionHist 30 : 6 OLD: -1.753254e+05 NEW: 1.544627e+05
       FReactionHist 31 : 1 OLD: 1.269810e+05 NEW: -1.799456e+03
       FReactionHist 31 : 2 OLD: 8.251867e+03 NEW: -1.340564e+03
       FReactionHist 31 : 3 OLD: 2.069228e+06 NEW: 2.133255e+06
       FReactionHist 31 : 4 OLD: -1.302810e+05 NEW: 1.062250e+05
       FReactionHist 31 : 5 OLD: 2.630570e+06 NEW: -2.017202e+05
       FReactionHist 31 : 6 OLD: -1.923637e+05 NEW: 1.769948e+05
       FReactionHist 32 : 1 OLD: 1.167076e+05 NEW: -7.204097e+01
       FReactionHist 32 : 2 OLD: 1.051775e+04 NEW: -6.959604e+03
       FReactionHist 32 : 3 OLD: 1.866105e+06 NEW: 1.922703e+06
       FReactionHist 32 : 4 OLD: -1.376543e+05 NEW: 1.502795e+05
       FReactionHist 32 : 5 OLD: 2.619023e+06 NEW: -1.998747e+05
       FReactionHist 32 : 6 OLD: -2.250440e+05 NEW: 1.957361e+05
       FReactionHist 33 : 1 OLD: 1.055757e+05 NEW: 6.232601e+02
       FReactionHist 33 : 2 OLD: 1.276755e+04 NEW: -1.354147e+04
       FReactionHist 33 : 3 OLD: 1.707179e+06 NEW: 1.748291e+06
       FReactionHist 33 : 4 OLD: -1.436184e+05 NEW: 2.006327e+05
       FReactionHist 33 : 5 OLD: 2.593474e+06 NEW: -2.018983e+05
       FReactionHist 33 : 6 OLD: -2.657375e+05 NEW: 2.100394e+05
       FReactionHist 34 : 1 OLD: 9.385162e+04 NEW: 3.508059e+02
       FReactionHist 34 : 2 OLD: 1.134134e+04 NEW: -1.922843e+04
       FReactionHist 34 : 3 OLD: 1.575011e+06 NEW: 1.589913e+06
       FReactionHist 34 : 4 OLD: -1.368330e+05 NEW: 2.471289e+05
       FReactionHist 34 : 5 OLD: 2.576019e+06 NEW: -2.083186e+05
       FReactionHist 34 : 6 OLD: -3.041237e+05 NEW: 2.238743e+05
       FReactionHist 35 : 1 OLD: 8.637810e+04 NEW: 8.609665e+02
       FReactionHist 35 : 2 OLD: 6.853035e+03 NEW: -2.358843e+04
       FReactionHist 35 : 3 OLD: 1.411005e+06 NEW: 1.415745e+06
       FReactionHist 35 : 4 OLD: -1.185265e+05 NEW: 2.807289e+05
       FReactionHist 35 : 5 OLD: 2.604190e+06 NEW: -2.173939e+05
       FReactionHist 35 : 6 OLD: -3.320457e+05 NEW: 2.391342e+05
       FReactionHist 36 : 1 OLD: 9.490298e+04 NEW: 9.670794e+02
       FReactionHist 36 : 2 OLD: 2.280761e+03 NEW: -2.509760e+04
       FReactionHist 36 : 3 OLD: 1.238541e+06 NEW: 1.242135e+06
       FReactionHist 36 : 4 OLD: -1.021918e+05 NEW: 2.938792e+05
       FReactionHist 36 : 5 OLD: 2.724584e+06 NEW: -2.331971e+05
       FReactionHist 36 : 6 OLD: -3.432683e+05 NEW: 2.545939e+05
       FReactionHist 37 : 1 OLD: 1.268966e+05 NEW: -1.680119e+03
       FReactionHist 37 : 2 OLD: 1.078024e+03 NEW: -2.257584e+04
       FReactionHist 37 : 3 OLD: 1.135310e+06 NEW: 1.119643e+06
       FReactionHist 37 : 4 OLD: -1.033049e+05 NEW: 2.831283e+05
       FReactionHist 37 : 5 OLD: 2.944936e+06 NEW: -2.603292e+05
       FReactionHist 37 : 6 OLD: -3.360152e+05 NEW: 2.735105e+05
       FReactionHist 38 : 1 OLD: 1.668647e+05 NEW: -5.346950e+03
       FReactionHist 38 : 2 OLD: 4.840742e+03 NEW: -1.744741e+04
       FReactionHist 38 : 3 OLD: 1.110722e+06 NEW: 1.064543e+06
       FReactionHist 38 : 4 OLD: -1.257647e+05 NEW: 2.527021e+05
       FReactionHist 38 : 5 OLD: 3.191046e+06 NEW: -2.954417e+05
       FReactionHist 38 : 6 OLD: -3.134078e+05 NEW: 3.017271e+05
       FReactionHist 39 : 1 OLD: 1.856342e+05 NEW: -8.084929e+03
       FReactionHist 39 : 2 OLD: 1.001564e+04 NEW: -1.147011e+04
       FReactionHist 39 : 3 OLD: 1.133153e+06 NEW: 1.044055e+06
       FReactionHist 39 : 4 OLD: -1.569109e+05 NEW: 2.125861e+05
       FReactionHist 39 : 5 OLD: 3.362434e+06 NEW: -3.335351e+05
       FReactionHist 39 : 6 OLD: -2.778774e+05 NEW: 3.377124e+05
       FReactionHist 40 : 1 OLD: 1.800385e+05 NEW: -1.145845e+04
       FReactionHist 40 : 2 OLD: 1.141842e+04 NEW: -5.598128e+03
       FReactionHist 40 : 3 OLD: 1.211304e+06 NEW: 1.060091e+06
       FReactionHist 40 : 4 OLD: -1.805170e+05 NEW: 1.710677e+05
       FReactionHist 40 : 5 OLD: 3.441220e+06 NEW: -3.749752e+05
       FReactionHist 40 : 6 OLD: -2.376497e+05 NEW: 3.722366e+05
       FReactionHist 41 : 1 OLD: 1.760826e+05 NEW: -1.558453e+04
       FReactionHist 41 : 2 OLD: 9.209623e+03 NEW: -5.630461e+02
       FReactionHist 41 : 3 OLD: 1.361596e+06 NEW: 1.156904e+06
       FReactionHist 41 : 4 OLD: -1.971051e+05 NEW: 1.355391e+05
       FReactionHist 41 : 5 OLD: 3.502013e+06 NEW: -4.203682e+05
       FReactionHist 41 : 6 OLD: -2.110962e+05 NEW: 3.936996e+05
       FReactionHist 42 : 1 OLD: 1.915293e+05 NEW: -1.949073e+04
       FReactionHist 42 : 2 OLD: 9.189790e+03 NEW: 2.387889e+03
       FReactionHist 42 : 3 OLD: 1.564778e+06 NEW: 1.326950e+06
       FReactionHist 42 : 4 OLD: -2.244041e+05 NEW: 1.137071e+05
       FReactionHist 42 : 5 OLD: 3.603496e+06 NEW: -4.681771e+05
       FReactionHist 42 : 6 OLD: -2.116156e+05 NEW: 4.004581e+05
       FReactionHist 43 : 1 OLD: 2.134499e+05 NEW: -2.385424e+04
       FReactionHist 43 : 2 OLD: 1.493455e+04 NEW: 2.758429e+03
       FReactionHist 43 : 3 OLD: 1.747241e+06 NEW: 1.490180e+06
       FReactionHist 43 : 4 OLD: -2.725513e+05 NEW: 1.085861e+05
       FReactionHist 43 : 5 OLD: 3.718355e+06 NEW: -5.181046e+05
       FReactionHist 43 : 6 OLD: -2.328435e+05 NEW: 4.041936e+05
       FReactionHist 44 : 1 OLD: 2.279972e+05 NEW: -2.909923e+04
       FReactionHist 44 : 2 OLD: 2.318336e+04 NEW: 9.873765e+02
       FReactionHist 44 : 3 OLD: 1.852176e+06 NEW: 1.599336e+06
       FReactionHist 44 : 4 OLD: -3.291470e+05 NEW: 1.195365e+05
       FReactionHist 44 : 5 OLD: 3.794033e+06 NEW: -5.690697e+05
       FReactionHist 44 : 6 OLD: -2.581807e+05 NEW: 4.148824e+05
       FReactionHist 45 : 1 OLD: 2.346854e+05 NEW: -3.333152e+04
       FReactionHist 45 : 2 OLD: 2.748708e+04 NEW: -1.976397e+03
       FReactionHist 45 : 3 OLD: 1.926817e+06 NEW: 1.694252e+06
       FReactionHist 45 : 4 OLD: -3.737321e+05 NEW: 1.432673e+05
       FReactionHist 45 : 5 OLD: 3.795662e+06 NEW: -6.160541e+05
       FReactionHist 45 : 6 OLD: -2.786692e+05 NEW: 4.306773e+05
       FReactionHist 46 : 1 OLD: 2.270696e+05 NEW: -3.663921e+04
       FReactionHist 46 : 2 OLD: 2.728351e+04 NEW: -5.935723e+03
       FReactionHist 46 : 3 OLD: 2.027003e+06 NEW: 1.814859e+06
       FReactionHist 46 : 4 OLD: -4.032465e+05 NEW: 1.746987e+05
       FReactionHist 46 : 5 OLD: 3.693849e+06 NEW: -6.563089e+05
       FReactionHist 46 : 6 OLD: -3.032877e+05 NEW: 4.486238e+05
       FReactionHist 47 : 1 OLD: 1.968345e+05 NEW: -3.952768e+04
       FReactionHist 47 : 2 OLD: 2.843782e+04 NEW: -9.812695e+03
       FReactionHist 47 : 3 OLD: 2.136777e+06 NEW: 1.943521e+06
       FReactionHist 47 : 4 OLD: -4.331840e+05 NEW: 2.082615e+05
       FReactionHist 47 : 5 OLD: 3.495431e+06 NEW: -6.870345e+05
       FReactionHist 47 : 6 OLD: -3.480500e+05 NEW: 4.723594e+05
       FReactionHist 48 : 1 OLD: 1.658532e+05 NEW: -4.001245e+04
       FReactionHist 48 : 2 OLD: 3.391308e+04 NEW: -1.227711e+04
       FReactionHist 48 : 3 OLD: 2.234898e+06 NEW: 2.059467e+06
       FReactionHist 48 : 4 OLD: -4.698479e+05 NEW: 2.370570e+05
       FReactionHist 48 : 5 OLD: 3.270410e+06 NEW: -7.030202e+05
       FReactionHist 48 : 6 OLD: -4.089249e+05 NEW: 5.038016e+05
       FReactionHist 49 : 1 OLD: 1.534348e+05 NEW: -3.802120e+04
       FReactionHist 49 : 2 OLD: 3.852997e+04 NEW: -1.280880e+04
       FReactionHist 49 : 3 OLD: 2.328983e+06 NEW: 2.179821e+06
       FReactionHist 49 : 4 OLD: -4.978654e+05 NEW: 2.582991e+05
       FReactionHist 49 : 5 OLD: 3.081498e+06 NEW: -7.043960e+05
       FReactionHist 49 : 6 OLD: -4.587724e+05 NEW: 5.380217e+05
       FReactionHist 50 : 1 OLD: 1.448550e+05 NEW: -3.580784e+04
       FReactionHist 50 : 2 OLD: 3.742459e+04 NEW: -1.249169e+04
       FReactionHist 50 : 3 OLD: 2.429362e+06 NEW: 2.316764e+06
       FReactionHist 50 : 4 OLD: -5.026903e+05 NEW: 2.763386e+05
       FReactionHist 50 : 5 OLD: 2.915128e+06 NEW: -6.945677e+05
       FReactionHist 50 : 6 OLD: -4.778675e+05 NEW: 5.726736e+05
       FReactionHist 51 : 1 OLD: 1.276172e+05 NEW: -3.232606e+04
       FReactionHist 51 : 2 OLD: 3.283271e+04 NEW: -1.309312e+04
       FReactionHist 51 : 3 OLD: 2.543811e+06 NEW: 2.464351e+06
       FReactionHist 51 : 4 OLD: -4.905020e+05 NEW: 2.962572e+05
       FReactionHist 51 : 5 OLD: 2.737870e+06 NEW: -6.742278e+05
       FReactionHist 51 : 6 OLD: -4.717114e+05 NEW: 6.094733e+05
       
Reversing all of the forces gives a much better answer.  I also used the actual normal force instead of just the radial:

       FReactionHist 2 : 1 OLD: -7.142141e+02 NEW: -1.947521e+02
       FReactionHist 2 : 2 OLD: -1.076061e+02 NEW: -1.820817e+02
       FReactionHist 2 : 3 OLD: 1.515958e+05 NEW: 1.503001e+05
       FReactionHist 2 : 4 OLD: 7.069546e+01 NEW: -3.807510e+02
       FReactionHist 2 : 5 OLD: -7.420892e+02 NEW: -8.080571e+01
       FReactionHist 2 : 6 OLD: 1.359528e+02 NEW: 5.150698e+01
       FReactionHist 3 : 1 OLD: -3.359312e+03 NEW: -7.015090e+02
       FReactionHist 3 : 2 OLD: -4.588144e+02 NEW: -2.124183e+02
       FReactionHist 3 : 3 OLD: 4.326783e+05 NEW: 4.244038e+05
       FReactionHist 3 : 4 OLD: -1.020519e+03 NEW: -5.012720e+03
       FReactionHist 3 : 5 OLD: 3.280347e+03 NEW: 2.363984e+03
       FReactionHist 3 : 6 OLD: 1.258519e+03 NEW: 2.050149e+02
       FReactionHist 4 : 1 OLD: -3.979357e+03 NEW: 3.491550e+02
       FReactionHist 4 : 2 OLD: -1.340640e+02 NEW: 1.789650e+03
       FReactionHist 4 : 3 OLD: 7.319887e+05 NEW: 7.125394e+05
       FReactionHist 4 : 4 OLD: -6.307394e+03 NEW: -1.727543e+04
       FReactionHist 4 : 5 OLD: 2.314380e+04 NEW: 1.126457e+04
       FReactionHist 4 : 6 OLD: 3.744007e+03 NEW: -1.270150e+03
       FReactionHist 5 : 1 OLD: -1.750950e+03 NEW: 2.271782e+03
       FReactionHist 5 : 2 OLD: 1.599643e+03 NEW: 4.498841e+03
       FReactionHist 5 : 3 OLD: 9.992354e+05 NEW: 9.746937e+05
       FReactionHist 5 : 4 OLD: -1.744892e+04 NEW: -3.285085e+04
       FReactionHist 5 : 5 OLD: 6.147387e+04 NEW: 2.351490e+04
       FReactionHist 5 : 6 OLD: 3.374528e+03 NEW: -8.112733e+03
       FReactionHist 6 : 1 OLD: 2.905970e+03 NEW: 2.849415e+03
       FReactionHist 6 : 2 OLD: 4.352465e+03 NEW: 5.854438e+03
       FReactionHist 6 : 3 OLD: 1.273110e+06 NEW: 1.245686e+06
       FReactionHist 6 : 4 OLD: -3.212057e+04 NEW: -4.598610e+04
       FReactionHist 6 : 5 OLD: 1.179283e+05 NEW: 3.536263e+04
       FReactionHist 6 : 6 OLD: -2.245956e+03 NEW: -1.587965e+04
       FReactionHist 7 : 1 OLD: 9.451359e+03 NEW: 3.274258e+03
       FReactionHist 7 : 2 OLD: 6.298100e+03 NEW: 6.857191e+03
       FReactionHist 7 : 3 OLD: 1.535036e+06 NEW: 1.502384e+06
       FReactionHist 7 : 4 OLD: -4.252528e+04 NEW: -5.507631e+04
       FReactionHist 7 : 5 OLD: 1.766629e+05 NEW: 4.543506e+04
       FReactionHist 7 : 6 OLD: -9.761377e+03 NEW: -1.937159e+04
       FReactionHist 8 : 1 OLD: 7.140729e+03 NEW: 3.472187e+03
       FReactionHist 8 : 2 OLD: 5.190312e+03 NEW: 6.687468e+03
       FReactionHist 8 : 3 OLD: 1.800449e+06 NEW: 1.759943e+06
       FReactionHist 8 : 4 OLD: -4.293471e+04 NEW: -5.717579e+04
       FReactionHist 8 : 5 OLD: 2.083231e+05 NEW: 5.400061e+04
       FReactionHist 8 : 6 OLD: -1.689814e+04 NEW: -2.294156e+04
       FReactionHist 9 : 1 OLD: -5.777089e+03 NEW: 2.963532e+03
       FReactionHist 9 : 2 OLD: 3.081757e+03 NEW: 4.616580e+03
       FReactionHist 9 : 3 OLD: 2.059662e+06 NEW: 2.005378e+06
       FReactionHist 9 : 4 OLD: -4.110436e+04 NEW: -5.409477e+04
       FReactionHist 9 : 5 OLD: 2.195080e+05 NEW: 6.113520e+04
       FReactionHist 9 : 6 OLD: -2.325660e+04 NEW: -2.887910e+04
       FReactionHist 10 : 1 OLD: -1.532398e+04 NEW: 2.112791e+03
       FReactionHist 10 : 2 OLD: 4.241030e+03 NEW: 3.087616e+03
       FReactionHist 10 : 3 OLD: 2.292043e+06 NEW: 2.228209e+06
       FReactionHist 10 : 4 OLD: -4.820738e+04 NEW: -5.309619e+04
       FReactionHist 10 : 5 OLD: 2.564952e+05 NEW: 6.635832e+04
       FReactionHist 10 : 6 OLD: -2.816460e+04 NEW: -3.613959e+04
       FReactionHist 11 : 1 OLD: -8.625607e+03 NEW: 1.383357e+03
       FReactionHist 11 : 2 OLD: 7.871172e+03 NEW: 3.834315e+03
       FReactionHist 11 : 3 OLD: 2.496174e+06 NEW: 2.433206e+06
       FReactionHist 11 : 4 OLD: -6.118161e+04 NEW: -5.733858e+04
       FReactionHist 11 : 5 OLD: 3.693550e+05 NEW: 7.510366e+04
       FReactionHist 11 : 6 OLD: -3.092693e+04 NEW: -4.368077e+04
       FReactionHist 12 : 1 OLD: 1.378745e+04 NEW: 1.909083e+03
       FReactionHist 12 : 2 OLD: 8.676335e+03 NEW: 4.405622e+03
       FReactionHist 12 : 3 OLD: 2.661933e+06 NEW: 2.609826e+06
       FReactionHist 12 : 4 OLD: -6.678975e+04 NEW: -6.583565e+04
       FReactionHist 12 : 5 OLD: 5.642306e+05 NEW: 9.253857e+04
       FReactionHist 12 : 6 OLD: -3.419050e+04 NEW: -5.193917e+04
       FReactionHist 13 : 1 OLD: 4.188317e+04 NEW: 4.602138e+03
       FReactionHist 13 : 2 OLD: 5.949264e+03 NEW: 4.949109e+03
       FReactionHist 13 : 3 OLD: 2.809592e+06 NEW: 2.779705e+06
       FReactionHist 13 : 4 OLD: -6.122785e+04 NEW: -7.505094e+04
       FReactionHist 13 : 5 OLD: 7.879436e+05 NEW: 1.216699e+05
       FReactionHist 13 : 6 OLD: -4.106554e+04 NEW: -6.306602e+04
       FReactionHist 14 : 1 OLD: 5.831440e+04 NEW: 7.627373e+03
       FReactionHist 14 : 2 OLD: 3.728332e+03 NEW: 6.052905e+03
       FReactionHist 14 : 3 OLD: 2.946149e+06 NEW: 2.946645e+06
       FReactionHist 14 : 4 OLD: -5.653142e+04 NEW: -8.601972e+04
       FReactionHist 14 : 5 OLD: 9.839289e+05 NEW: 1.616642e+05
       FReactionHist 14 : 6 OLD: -5.029699e+04 NEW: -7.733837e+04
       FReactionHist 15 : 1 OLD: 6.152321e+04 NEW: 1.103870e+04
       FReactionHist 15 : 2 OLD: 4.844859e+03 NEW: 7.526892e+03
       FReactionHist 15 : 3 OLD: 3.057770e+06 NEW: 3.091675e+06
       FReactionHist 15 : 4 OLD: -6.372762e+04 NEW: -1.015444e+05
       FReactionHist 15 : 5 OLD: 1.143327e+06 NEW: 2.140174e+05
       FReactionHist 15 : 6 OLD: -6.014461e+04 NEW: -9.251724e+04
       FReactionHist 16 : 1 OLD: 6.731060e+04 NEW: 1.767837e+04
       FReactionHist 16 : 2 OLD: 8.376310e+03 NEW: 1.052916e+04
       FReactionHist 16 : 3 OLD: 3.153126e+06 NEW: 3.216274e+06
       FReactionHist 16 : 4 OLD: -7.872475e+04 NEW: -1.186444e+05
       FReactionHist 16 : 5 OLD: 1.305007e+06 NEW: 2.855351e+05
       FReactionHist 16 : 6 OLD: -7.497424e+04 NEW: -1.086858e+05
       FReactionHist 17 : 1 OLD: 8.230119e+04 NEW: 2.777991e+04
       FReactionHist 17 : 2 OLD: 1.024532e+04 NEW: 1.283029e+04
       FReactionHist 17 : 3 OLD: 3.230910e+06 NEW: 3.313792e+06
       FReactionHist 17 : 4 OLD: -8.822670e+04 NEW: -1.316353e+05
       FReactionHist 17 : 5 OLD: 1.510418e+06 NEW: 3.751819e+05
       FReactionHist 17 : 6 OLD: -9.765531e+04 NEW: -1.265610e+05
       FReactionHist 18 : 1 OLD: 1.065081e+05 NEW: 3.873575e+04
       FReactionHist 18 : 2 OLD: 7.938057e+03 NEW: 1.220816e+04
       FReactionHist 18 : 3 OLD: 3.273443e+06 NEW: 3.376282e+06
       FReactionHist 18 : 4 OLD: -8.456955e+04 NEW: -1.340582e+05
       FReactionHist 18 : 5 OLD: 1.764005e+06 NEW: 4.703257e+05
       FReactionHist 18 : 6 OLD: -1.227681e+05 NEW: -1.473060e+05
       FReactionHist 19 : 1 OLD: 1.394250e+05 NEW: 4.767486e+04
       FReactionHist 19 : 2 OLD: 4.080830e+03 NEW: 9.696925e+03
       FReactionHist 19 : 3 OLD: 3.285289e+06 NEW: 3.419982e+06
       FReactionHist 19 : 4 OLD: -7.720880e+04 NEW: -1.267976e+05
       FReactionHist 19 : 5 OLD: 2.048196e+06 NEW: 5.564637e+05
       FReactionHist 19 : 6 OLD: -1.418991e+05 NEW: -1.699968e+05
       FReactionHist 20 : 1 OLD: 1.754878e+05 NEW: 5.391416e+04
       FReactionHist 20 : 2 OLD: 4.148639e+03 NEW: 7.669312e+03
       FReactionHist 20 : 3 OLD: 3.268342e+06 NEW: 3.445025e+06
       FReactionHist 20 : 4 OLD: -8.418677e+04 NEW: -1.207445e+05
       FReactionHist 20 : 5 OLD: 2.308836e+06 NEW: 6.229136e+05
       FReactionHist 20 : 6 OLD: -1.498487e+05 NEW: -1.905319e+05
       FReactionHist 21 : 1 OLD: 1.920116e+05 NEW: 5.694175e+04
       FReactionHist 21 : 2 OLD: 9.263111e+03 NEW: 7.111259e+03
       FReactionHist 21 : 3 OLD: 3.214659e+06 NEW: 3.428267e+06
       FReactionHist 21 : 4 OLD: -1.081526e+05 NEW: -1.191782e+05
       FReactionHist 21 : 5 OLD: 2.448197e+06 NEW: 6.635710e+05
       FReactionHist 21 : 6 OLD: -1.464631e+05 NEW: -2.068825e+05
       FReactionHist 22 : 1 OLD: 1.718877e+05 NEW: 5.656224e+04
       FReactionHist 22 : 2 OLD: 1.438798e+04 NEW: 7.127622e+03
       FReactionHist 22 : 3 OLD: 3.118671e+06 NEW: 3.357455e+06
       FReactionHist 22 : 4 OLD: -1.320333e+05 NEW: -1.205401e+05
       FReactionHist 22 : 5 OLD: 2.423031e+06 NEW: 6.743967e+05
       FReactionHist 22 : 6 OLD: -1.365775e+05 NEW: -2.181784e+05
       FReactionHist 23 : 1 OLD: 1.338156e+05 NEW: 5.197891e+04
       FReactionHist 23 : 2 OLD: 1.428200e+04 NEW: 6.467957e+03
       FReactionHist 23 : 3 OLD: 2.979634e+06 NEW: 3.240676e+06
       FReactionHist 23 : 4 OLD: -1.393290e+05 NEW: -1.243978e+05
       FReactionHist 23 : 5 OLD: 2.306570e+06 NEW: 6.539564e+05
       FReactionHist 23 : 6 OLD: -1.308751e+05 NEW: -2.269103e+05
       FReactionHist 24 : 1 OLD: 1.124152e+05 NEW: 4.387376e+04
       FReactionHist 24 : 2 OLD: 1.084510e+04 NEW: 6.971693e+03
       FReactionHist 24 : 3 OLD: 2.817431e+06 NEW: 3.105692e+06
       FReactionHist 24 : 4 OLD: -1.336725e+05 NEW: -1.308872e+05
       FReactionHist 24 : 5 OLD: 2.216584e+06 NEW: 6.071152e+05
       FReactionHist 24 : 6 OLD: -1.385456e+05 NEW: -2.365181e+05
       FReactionHist 25 : 1 OLD: 1.074521e+05 NEW: 3.384298e+04
       FReactionHist 25 : 2 OLD: 9.615636e+03 NEW: 8.785176e+03
       FReactionHist 25 : 3 OLD: 2.657899e+06 NEW: 2.977204e+06
       FReactionHist 25 : 4 OLD: -1.309875e+05 NEW: -1.388824e+05
       FReactionHist 25 : 5 OLD: 2.190722e+06 NEW: 5.439435e+05
       FReactionHist 25 : 6 OLD: -1.564494e+05 NEW: -2.445604e+05
       FReactionHist 26 : 1 OLD: 9.848444e+04 NEW: 2.355010e+04
       FReactionHist 26 : 2 OLD: 1.120493e+04 NEW: 8.703740e+03
       FReactionHist 26 : 3 OLD: 2.527702e+06 NEW: 2.869005e+06
       FReactionHist 26 : 4 OLD: -1.361058e+05 NEW: -1.383197e+05
       FReactionHist 26 : 5 OLD: 2.196934e+06 NEW: 4.760863e+05
       FReactionHist 26 : 6 OLD: -1.715531e+05 NEW: -2.477193e+05
       FReactionHist 27 : 1 OLD: 9.141744e+04 NEW: 1.441772e+04
       FReactionHist 27 : 2 OLD: 1.228409e+04 NEW: 4.607069e+03
       FReactionHist 27 : 3 OLD: 2.462574e+06 NEW: 2.802921e+06
       FReactionHist 27 : 4 OLD: -1.417384e+05 NEW: -1.238040e+05
       FReactionHist 27 : 5 OLD: 2.244657e+06 NEW: 4.144337e+05
       FReactionHist 27 : 6 OLD: -1.768987e+05 NEW: -2.498816e+05
       FReactionHist 28 : 1 OLD: 1.038709e+05 NEW: 7.326526e+03
       FReactionHist 28 : 2 OLD: 1.210098e+04 NEW: -8.375149e+02
       FReactionHist 28 : 3 OLD: 2.444617e+06 NEW: 2.763663e+06
       FReactionHist 28 : 4 OLD: -1.425837e+05 NEW: -1.072385e+05
       FReactionHist 28 : 5 OLD: 2.359731e+06 NEW: 3.678111e+05
       FReactionHist 28 : 6 OLD: -1.750634e+05 NEW: -2.563501e+05
       FReactionHist 29 : 1 OLD: 1.249837e+05 NEW: 3.748226e+03
       FReactionHist 29 : 2 OLD: 1.114729e+04 NEW: -3.559884e+03
       FReactionHist 29 : 3 OLD: 2.393481e+06 NEW: 2.688654e+06
       FReactionHist 29 : 4 OLD: -1.380475e+05 NEW: -1.040723e+05
       FReactionHist 29 : 5 OLD: 2.501623e+06 NEW: 3.411824e+05
       FReactionHist 29 : 6 OLD: -1.721517e+05 NEW: -2.718706e+05
       FReactionHist 30 : 1 OLD: 1.322929e+05 NEW: 2.898737e+03
       FReactionHist 30 : 2 OLD: 9.029000e+03 NEW: -3.119338e+03
       FReactionHist 30 : 3 OLD: 2.263661e+06 NEW: 2.538781e+06
       FReactionHist 30 : 4 OLD: -1.313334e+05 NEW: -1.244353e+05
       FReactionHist 30 : 5 OLD: 2.599713e+06 NEW: 3.306308e+05
       FReactionHist 30 : 6 OLD: -1.753254e+05 NEW: -2.983760e+05
       FReactionHist 31 : 1 OLD: 1.269810e+05 NEW: 1.624383e+03
       FReactionHist 31 : 2 OLD: 8.251867e+03 NEW: 1.747869e+03
       FReactionHist 31 : 3 OLD: 2.069228e+06 NEW: 2.317415e+06
       FReactionHist 31 : 4 OLD: -1.302810e+05 NEW: -1.740880e+05
       FReactionHist 31 : 5 OLD: 2.630570e+06 NEW: 3.251402e+05
       FReactionHist 31 : 6 OLD: -1.923637e+05 NEW: -3.351223e+05
       FReactionHist 32 : 1 OLD: 1.167076e+05 NEW: -1.266819e+03
       FReactionHist 32 : 2 OLD: 1.051775e+04 NEW: 1.124192e+04
       FReactionHist 32 : 3 OLD: 1.866105e+06 NEW: 2.075282e+06
       FReactionHist 32 : 4 OLD: -1.376543e+05 NEW: -2.480941e+05
       FReactionHist 32 : 5 OLD: 2.619023e+06 NEW: 3.213116e+05
       FReactionHist 32 : 6 OLD: -2.250440e+05 NEW: -3.805348e+05
       FReactionHist 33 : 1 OLD: 1.055757e+05 NEW: -2.337332e+03
       FReactionHist 33 : 2 OLD: 1.276755e+04 NEW: 2.233258e+04
       FReactionHist 33 : 3 OLD: 1.707179e+06 NEW: 1.867914e+06
       FReactionHist 33 : 4 OLD: -1.436184e+05 NEW: -3.325593e+05
       FReactionHist 33 : 5 OLD: 2.593474e+06 NEW: 3.244220e+05
       FReactionHist 33 : 6 OLD: -2.657375e+05 NEW: -4.313688e+05
       FReactionHist 34 : 1 OLD: 9.385162e+04 NEW: -1.730156e+03
       FReactionHist 34 : 2 OLD: 1.134134e+04 NEW: 3.188432e+04
       FReactionHist 34 : 3 OLD: 1.575011e+06 NEW: 1.683719e+06
       FReactionHist 34 : 4 OLD: -1.368330e+05 NEW: -4.103471e+05
       FReactionHist 34 : 5 OLD: 2.576019e+06 NEW: 3.350594e+05
       FReactionHist 34 : 6 OLD: -3.041237e+05 NEW: -4.835886e+05
       FReactionHist 35 : 1 OLD: 8.637810e+04 NEW: -2.486678e+03
       FReactionHist 35 : 2 OLD: 6.853035e+03 NEW: 3.919765e+04
       FReactionHist 35 : 3 OLD: 1.411005e+06 NEW: 1.481692e+06
       FReactionHist 35 : 4 OLD: -1.185265e+05 NEW: -4.662377e+05
       FReactionHist 35 : 5 OLD: 2.604190e+06 NEW: 3.498219e+05
       FReactionHist 35 : 6 OLD: -3.320457e+05 NEW: -5.296568e+05
       FReactionHist 36 : 1 OLD: 9.490298e+04 NEW: -2.649555e+03
       FReactionHist 36 : 2 OLD: 2.280761e+03 NEW: 4.170122e+04
       FReactionHist 36 : 3 OLD: 1.238541e+06 NEW: 1.282924e+06
       FReactionHist 36 : 4 OLD: -1.021918e+05 NEW: -4.874704e+05
       FReactionHist 36 : 5 OLD: 2.724584e+06 NEW: 3.751963e+05
       FReactionHist 36 : 6 OLD: -3.432683e+05 NEW: -5.595231e+05
       FReactionHist 37 : 1 OLD: 1.268966e+05 NEW: 1.738349e+03
       FReactionHist 37 : 2 OLD: 1.078024e+03 NEW: 3.737133e+04
       FReactionHist 37 : 3 OLD: 1.135310e+06 NEW: 1.143732e+06
       FReactionHist 37 : 4 OLD: -1.033049e+05 NEW: -4.680846e+05
       FReactionHist 37 : 5 OLD: 2.944936e+06 NEW: 4.186495e+05
       FReactionHist 37 : 6 OLD: -3.360152e+05 NEW: -5.707338e+05
       FReactionHist 38 : 1 OLD: 1.668647e+05 NEW: 7.698624e+03
       FReactionHist 38 : 2 OLD: 4.840742e+03 NEW: 2.859188e+04
       FReactionHist 38 : 3 OLD: 1.110722e+06 NEW: 1.067275e+06
       FReactionHist 38 : 4 OLD: -1.257647e+05 NEW: -4.150855e+05
       FReactionHist 38 : 5 OLD: 3.191046e+06 NEW: 4.742342e+05
       FReactionHist 38 : 6 OLD: -3.134078e+05 NEW: -5.673061e+05
       FReactionHist 39 : 1 OLD: 1.856342e+05 NEW: 1.189944e+04
       FReactionHist 39 : 2 OLD: 1.001564e+04 NEW: 1.832652e+04
       FReactionHist 39 : 3 OLD: 1.133153e+06 NEW: 1.021746e+06
       FReactionHist 39 : 4 OLD: -1.569109e+05 NEW: -3.452624e+05
       FReactionHist 39 : 5 OLD: 3.362434e+06 NEW: 5.333778e+05
       FReactionHist 39 : 6 OLD: -2.778774e+05 NEW: -5.509971e+05
       FReactionHist 40 : 1 OLD: 1.800385e+05 NEW: 1.702315e+04
       FReactionHist 40 : 2 OLD: 1.141842e+04 NEW: 8.176275e+03
       FReactionHist 40 : 3 OLD: 1.211304e+06 NEW: 1.022252e+06
       FReactionHist 40 : 4 OLD: -1.805170e+05 NEW: -2.725721e+05
       FReactionHist 40 : 5 OLD: 3.441220e+06 NEW: 5.968722e+05
       FReactionHist 40 : 6 OLD: -2.376497e+05 NEW: -5.285826e+05
       FReactionHist 41 : 1 OLD: 1.760826e+05 NEW: 2.334977e+04
       FReactionHist 41 : 2 OLD: 9.209623e+03 NEW: -6.175105e+02
       FReactionHist 41 : 3 OLD: 1.361596e+06 NEW: 1.107408e+06
       FReactionHist 41 : 4 OLD: -1.971051e+05 NEW: -2.095572e+05
       FReactionHist 41 : 5 OLD: 3.502013e+06 NEW: 6.661509e+05
       FReactionHist 41 : 6 OLD: -2.110962e+05 NEW: -5.108955e+05
       FReactionHist 42 : 1 OLD: 1.915293e+05 NEW: 2.928337e+04
       FReactionHist 42 : 2 OLD: 9.189790e+03 NEW: -5.928079e+03
       FReactionHist 42 : 3 OLD: 1.564778e+06 NEW: 1.264525e+06
       FReactionHist 42 : 4 OLD: -2.244041e+05 NEW: -1.693958e+05
       FReactionHist 42 : 5 OLD: 3.603496e+06 NEW: 7.390725e+05
       FReactionHist 42 : 6 OLD: -2.116156e+05 NEW: -5.045395e+05
       FReactionHist 43 : 1 OLD: 2.134499e+05 NEW: 3.603316e+04
       FReactionHist 43 : 2 OLD: 1.493455e+04 NEW: -6.872088e+03
       FReactionHist 43 : 3 OLD: 1.747241e+06 NEW: 1.420253e+06
       FReactionHist 43 : 4 OLD: -2.725513e+05 NEW: -1.574298e+05
       FReactionHist 43 : 5 OLD: 3.718355e+06 NEW: 8.155610e+05
       FReactionHist 43 : 6 OLD: -2.328435e+05 NEW: -5.096905e+05
       FReactionHist 44 : 1 OLD: 2.279972e+05 NEW: 4.435607e+04
       FReactionHist 44 : 2 OLD: 2.318336e+04 NEW: -4.159295e+03
       FReactionHist 44 : 3 OLD: 1.852176e+06 NEW: 1.525401e+06
       FReactionHist 44 : 4 OLD: -3.291470e+05 NEW: -1.728541e+05
       FReactionHist 44 : 5 OLD: 3.794033e+06 NEW: 8.941186e+05
       FReactionHist 44 : 6 OLD: -2.581807e+05 NEW: -5.243099e+05
       FReactionHist 45 : 1 OLD: 2.346854e+05 NEW: 5.106663e+04
       FReactionHist 45 : 2 OLD: 2.748708e+04 NEW: 6.422212e+02
       FReactionHist 45 : 3 OLD: 1.926817e+06 NEW: 1.617446e+06
       FReactionHist 45 : 4 OLD: -3.737321e+05 NEW: -2.103492e+05
       FReactionHist 45 : 5 OLD: 3.795662e+06 NEW: 9.664687e+05
       FReactionHist 45 : 6 OLD: -2.786692e+05 NEW: -5.492553e+05
       FReactionHist 46 : 1 OLD: 2.270696e+05 NEW: 5.632689e+04
       FReactionHist 46 : 2 OLD: 2.728351e+04 NEW: 7.225037e+03
       FReactionHist 46 : 3 OLD: 2.027003e+06 NEW: 1.744027e+06
       FReactionHist 46 : 4 OLD: -4.032465e+05 NEW: -2.615504e+05
       FReactionHist 46 : 5 OLD: 3.693849e+06 NEW: 1.028030e+06
       FReactionHist 46 : 6 OLD: -3.032877e+05 NEW: -5.911100e+05
       FReactionHist 47 : 1 OLD: 1.968345e+05 NEW: 6.099089e+04
       FReactionHist 47 : 2 OLD: 2.843782e+04 NEW: 1.381642e+04
       FReactionHist 47 : 3 OLD: 2.136777e+06 NEW: 1.891374e+06
       FReactionHist 47 : 4 OLD: -4.331840e+05 NEW: -3.171665e+05
       FReactionHist 47 : 5 OLD: 3.495431e+06 NEW: 1.073952e+06
       FReactionHist 47 : 6 OLD: -3.480500e+05 NEW: -6.539719e+05
       FReactionHist 48 : 1 OLD: 1.658532e+05 NEW: 6.163925e+04
       FReactionHist 48 : 2 OLD: 3.391308e+04 NEW: 1.815692e+04
       FReactionHist 48 : 3 OLD: 2.234898e+06 NEW: 2.035748e+06
       FReactionHist 48 : 4 OLD: -4.698479e+05 NEW: -3.653487e+05
       FReactionHist 48 : 5 OLD: 3.270410e+06 NEW: 1.095111e+06
       FReactionHist 48 : 6 OLD: -4.089249e+05 NEW: -7.312105e+05
       FReactionHist 49 : 1 OLD: 1.534348e+05 NEW: 5.809436e+04
       FReactionHist 49 : 2 OLD: 3.852997e+04 NEW: 1.926616e+04
       FReactionHist 49 : 3 OLD: 2.328983e+06 NEW: 2.192538e+06
       FReactionHist 49 : 4 OLD: -4.978654e+05 NEW: -4.008585e+05
       FReactionHist 49 : 5 OLD: 3.081498e+06 NEW: 1.091335e+06
       FReactionHist 49 : 6 OLD: -4.587724e+05 NEW: -8.077275e+05
       FReactionHist 50 : 1 OLD: 1.448550e+05 NEW: 5.414408e+04
       FReactionHist 50 : 2 OLD: 3.742459e+04 NEW: 1.887549e+04
       FReactionHist 50 : 3 OLD: 2.429362e+06 NEW: 2.374931e+06
       FReactionHist 50 : 4 OLD: -5.026903e+05 NEW: -4.303957e+05
       FReactionHist 50 : 5 OLD: 2.915128e+06 NEW: 1.068029e+06
       FReactionHist 50 : 6 OLD: -4.778675e+05 NEW: -8.674547e+05
       FReactionHist 51 : 1 OLD: 1.276172e+05 NEW: 4.795919e+04
       FReactionHist 51 : 2 OLD: 3.283271e+04 NEW: 1.990488e+04
       FReactionHist 51 : 3 OLD: 2.543811e+06 NEW: 2.574724e+06
       FReactionHist 51 : 4 OLD: -4.905020e+05 NEW: -4.619791e+05
       FReactionHist 51 : 5 OLD: 2.737870e+06 NEW: 1.026000e+06
       FReactionHist 51 : 6 OLD: -4.717114e+05 NEW: -9.004793e+05